In [1]:
#
# Washington Multiple AGN (WMAGN) Catalog
# Author: R. W. Pfeifle
# Date Created: 18 June 2020
# Last Revision Date: 15 July 2020

# Purpose:
# 
# 
# 
# 
# 
# 

In [2]:
# Load in packages for pandas, astropy, etc. 

import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.table import Column, MaskedColumn
from astropy.io.ascii import masked

pd.set_option('display.max_rows', None) # Setting max number of rows per df to be the size of the df

--------
Summary:
--------
The purpose of this catalog is to have an exhaustive list of all known multiple
AGNs from the literature. A multiple AGN is defined as the manifestation of two
or more AGNs at some wavelength that are associated with each other due to some
astrophysical process. This astrophysical process can be:

- Multiple AGNs in a galaxy interaction, merger, or post-merger
- Multiple AGNs that are gravitationally bound, e.g. binary AGNs
- The appearance of multiple AGNs due to gravitational lensing (e.g. an Einstein cross)

This catalog should be *exhaustive*, so include not only confirmed systems but
also candidate systems. These systems should be everything that indicates or
potentially indicates the presence or apparent presence of more than one AGN
associated with a system (e.g. multiple X-ray/radio component, doubling of
spectroscopic lines, astrometric jitter, etc.)

----------------------
WMAGN catalog columns:
----------------------

System type (Dual/triple AGN, binary AGN, or gravitational lens)

Discovery method (e.g., multiple X-ray/radio components, spectroscopic, astrometric, etc.)

Literature name (e.g. SDSS J084905.51+111447.2)

Component (e.g. Gal1/Gal2/Gal3, a/b/c, 1/2/3, etc)
- One row for each component; components share same unique system name

Unique, standardized system name
- NED, Simbad, or IAU name preferred

Coordinates
- Must be in ICRS system. If listed in FK5/FK6, convert to ICRS (e.g. with AstroPy)
- Equinox J2000.0
- Native epoch (i.e. not corrected for "proper motion")
- Degrees, full precision (i.e. 0.00001 deg = 0.036 arcsec)
- If coordinates not available for each component individually, just repeat the
  listed system coordinates for each source.

Coordinate epoch (e.g. 2015.5 for Gaia DR2)
- Leave blank if not determinable

Coordinate wavelength/frequency
- Leave blank if not determinable

Coordinate source (e.g. SDSS, 2MASS, VLBA, Chandra, etc)

Redshift
- Leave blank if not determinable

Redshift type (spec/photo)
- Leave blank if not determinable

Brightness band
- E.g., Johnson V, SDSS r, Gaia G, WISE W1, radio X-band, etc.

Brightness numerical value
- Magnitude or flux density

Brightness type
- E.g., VEGAMAG, ABMAG/STMAG, Jy

Bibcode(s), e.g. 2019ApJ...883..167P,2019ApJ...887...90L
- Ordered by date

NED hyperlink

Simbad hyperlink

Confidence level
- Up to you how you want to qualify this. E.g., "potential", "likely", "confirmed"
- Preferably numeric so catalog can be sorted. E.g. 1=potential, 2=likely, 3=confirmed.

Essential object notes


In [3]:
# Loading in Xin Liu's 2011 catalog of optically selected AGN pairs here...

#liu_pairs = (ascii.read('XLiu2011/agnpairs.dat')).to_pandas()


ReadME header from Liu's file:
-----

Title: Active Galactic Nucleus Pairs from the Sloan Digital Sky Survey.  I. 
       The Frequency on ~5-100 kpc Scales  
Authors: Liu X., Shen Y., Strauss M.A., Hao L.

Table: SDSS AGN Pairs

Byte-by-byte Description of file: apj398354t1_mrt.txt

Bytes Format Units  Label   Explanations

------------------------------
-   1- 19 A19    ---    Name    SDSS Designation (1)
-  21- 24 I4     ---    Plate   Spectroscopic plate number
-  26- 28 I3     ---    Fiber   Fiber identification
-  30- 34 I5     d      MJD     Modified Julian Date of observation
-  36- 41 F6.4   ---    z       Redshift
-  43- 47 F5.1   arcsec DelTh   Angular separation
-  49- 52 F4.1   kpc    rp      Transverse proper separation; h_70_^-1^ kpc
-  54- 56 I3     km/s   DelV    Line-of-sight velocity offset
-  58- 62 F5.2   mag    rmag    SDSS r-band model magnitude (2)
-      64 I1     ---    FAGN    Sub-population flag (3)
-      66 I1     ---    FTidal  Tidal flag (4)
--------------------------------------------------------------------------------
- Note (1): With J2000 coordinates given in the form of "hhmmss.ss+ddmmss.s".
- Note (2): Corrected for Galactic extinction.
- Note (3): Flag for sub-populations in the parent AGN sample. "0" through "2" are for narrow-line AGNs contained in the MPA-JHU DR7 catalog, where "0" stands for Seyferts, "1" for LINERs, and "2" for composites, respectively, according to the Kewley et al. (2001) and Kauffmann et al. (2003a) criteria for separating AGNs and composites  from H II regions, and the Ho et al. (1997) criterion for separating Seyferts from LINERS (Figure 4). "3" for narrow-line quasars in the Reyes et al. (2008) sample but not in the MPA-JHU DR7 catalog, "4" for broad-line quasars, and "5" for broad-line AGNs according to the Hao et al. (2005b) selection.
- Note (4): Flag from visual identification of tidal features. "0" for "non-interaction", "1" for ambiguous, "2" for "tidal", and "3" for "dumbbell" systems.
--------------------------------------------------------------------------------


In [4]:
#liu_pairs

In [5]:
#liu_pairs['Coordinates'] = liu_pairs['Name'].str.replace('J', '', regex=False)
#liu_pairs['Coordinates'] = liu_pairs['Coordinates'].str.replace('+', ' +', regex=False)
#liu_pairs['Coordinates'] = liu_pairs['Coordinates'].str.replace('-', ' -', regex=False)
#liu_pairs

In [5]:
# Creating the dataframe for the master catalog here, assigning the columns that we want (so far).
# Master catalog dataframe is referred to as 'kyber', as in 'kyber crystal'

dat = {'System_name':[], 'Literature_name':[], 'Component_number':[], 'Unique_name':[], 'System_type':[], \
       'Discovery_method':[], 'RA':[], 'Dec':[], 'Epoch':[], 'Coord_lambda_nu':[], 'Coord_source':[], \
       'Redshift':[], 'Redshift_type':[], 'Brightness_band':[], 'Brightness_val':[], 'Brightness_units':[], \
       'Bibcode':[], 'NED_link':[], 'SIMBAD_link':[], 'Confidence_level':[], 'Notes':[]}

kyber = pd.DataFrame(data=dat)



In [6]:
kyber

,System_name,Literature_name,Component_number,Unique_name,System_type,Discovery_method,RA,Dec,Epoch,Coord_lambda_nu,...,Redshift,Redshift_type,Brightness_band,Brightness_val,Brightness_units,Bibcode,NED_link,SIMBAD_link,Confidence_level,Notes


In [11]:
# Here tabulating the papers and sources from Satyapal's first authored papers, as well as her collaborators \
# Pfeifle, Ellison, Secrest, etc.

pfeif = pd.read_csv('pfeifle_duals.csv', delimiter=',')
#pfeif

Notes on Pfeifle+2019b, Pfeifle+2019a, Satyapal+2017, Ellison+2017 Triples/Candidates and Duals/Candidates
-----------------

Ellison+2017: https://doi.org/10.1093/mnrasl/slx076

Satyapal+2017: https://doi.org/10.3847/1538-4357/aa88ca

Pfeifle+2019a: https://doi.org/10.3847/1538-4357/ab07bc

Pfeifle+2019b: https://doi.org/10.3847/1538-4357/ab3a9b


Pfeifle+2019b examined the triple AGN XXXXX and came to the conclusion:
-------
- J0849

---------------------

Pfeifle+2019a examined 15 merging systems and came to the following conclusions:
-----
- J
- J
- J
- J
- J
- J
- J
- J
- J
- J
- J
- J
- J
- J
- J
---------------------

Satyapal+2017 examined 6 merging systems (which Pfeifle+2019a later re-examined and added to) and concluded:
----
- J
- J
- J
- J
- J
- J

---------------------

Ellison+2017 examined the merger XXXXX and concluded:
---
- J

----------


Because it is relevant to J1036, I will discuss the results of Dutta+2018 here:
------
Dutta+2018: https://doi.org/10.1093/mnras/sty1872

-J

In [ ]:
# Here tabulating the papers and sources for Comerford's first authored papers:
# Currently missing these columns from the current Comerford table: ...
# Epoch,Brightness_band,Brightness_val,Brightness_units,NED_link,SIMBAD_link,Confidence_level,Notes

comerford = {}

comerfordXXXX = {}

#comerford = comerford.append(new_row, ignore_index=True)


-----------------------
Notes on Comerford+2015 and Foord+2020 Duals/Candidates
----------------------

Comerford+2015: https://doi.org/10.1088/0004-637X/806/2/219

Foord+2020:  https://doi.org/10.3847/1538-4357/ab72fa


Comerford+2015 examined the following targets and came to the following conclusions: 
----------------------
- J0142−0050 --> Likely Single AGN
- J0752+2736 --> Likely Single AGN
- J0841+0101 --> Dual AGN/Offset AGN
- J0854+5026 --> Likely Single AGN
- J0952+2552 --> Dual AGN/Offset AGN
- J1006+4647 --> Likely Single AGN
- J1126+2944 --> Dual AGN
- J1239+5314 --> Dual AGN/Offset AGN
- J1322+2631 --> Dual AGN/Offset AGN
- J1356+1026 --> Dual AGN/Offset AGN
- J1448+1825 --> Likely Single AGN
- J1604+5009 --> Likely Single AGN

Foord+2020 looked at the same 12 targets and came up with the following conclusions:
-----------------------
- J0142−0050 --> Baymax does not favor a dual source
- J0752+2736 --> Dual AGN Candidate (two sources), conservatively cannot state is a dual AGN
- J0841+0101 --> Likely Single AGN
- J0854+5026 --> Baymax does not favor a dual source
- J0952+2552 --> Baymax does not favor a dual source
- J1006+4647 --> Baymax does not favor a dual source
- J1126+2944 --> Dual AGN
- J1239+5314 --> Baymax does not favor a dual source
- J1322+2631 --> Baymax does not favor a dual source
- J1356+1026 --> Dual AGN Candidate, conservatively not quoted as a dual AGN
- J1448+1825 --> Dual AGN Candidate, conservatively not quoted as a dual AGN
- J1604+5009 --> Baymax does not favor a dual source



We need to be mindful of the overlap between studies conducted within separate research groups as well as overlap between different research groups, as one can notice above...
----------



## Notes on Double-Peaked Optical Line Studies
------

Major (small initial or large systematic) studies which assembled candidate systems based strictly on double-peaked emission lines (these are not follow-up studies). These examine SDSS:


- Comerford+2009: (DEEP2 field) https://doi.org/10.1088/0004-637X/698/1/956 *(TABLES NOT DOWNLOADED)*
    - Title: INSPIRALLING SUPERMASSIVE BLACK HOLES: A NEW SIGNPOST FOR GALAXY MERGERS
        - Abstract: We present a new technique for observationally identifying galaxy mergers spectroscopically rather than through host galaxy imaging. Our technique exploits the dynamics of supermassive black holes (SMBHs) powering active galactic nuclei (AGNs) in merger-remnant galaxies. Because structure in the universe is built up through galaxy mergers and nearly all galaxies host a central SMBH, some galaxies should possess two SMBHs near their centers as the result of a recent merger. These SMBHs spiral to the center of the resultant merger-remnant galaxy, and one or both of the SMBHs may power AGNs. Using the DEEP2 Galaxy Redshift Survey, we have examined 1881 red galaxies, of which 91 exhibit [O III] and Hβ emission lines indicative of Seyfert 2 activity. Of these, 32 AGNs have [O III] emission-line redshifts significantly different from the redshifts of the host galaxies' stars, corresponding to velocity offsets of ~50 km s–1 to ~300 km s–1. Two of these AGNs exhibit double-peaked [O III] emission lines, while the remaining 30 AGNs each exhibit a single set of velocity-offset [O III] emission lines. After exploring a variety of physical models for these velocity offsets, we argue that the most likely explanation is inspiralling SMBHs in merger-remnant galaxies. Based on this interpretation, we find that roughly half of the red galaxies hosting AGNs are also merger remnants, which implies that mergers may trigger AGN activity in red galaxies. The AGN velocity offsets we find imply a merger fraction of ~30% and a merger rate of ~3 mergers Gyr–1 for red galaxies at redshifts 0.34 < z < 0.82.
        
        
- Wang+2009 (SDSS): https://doi.org/10.1088/0004-637X/705/1/L76 (Tables have been downloaded)
    - Title: ACTIVE GALACTIC NUCLEI WITH DOUBLE-PEAKED NARROW LINES: ARE THEY DUAL ACTIVE GALACTIC NUCLEI?
        - Abstract: Double-peaked [O III] profiles in active galactic nuclei (AGNs) may provide evidence for the existence of dual AGNs, but a good diagnostic for selecting them is currently lacking. Starting from ~7000 active galaxies in Sloan Digital Sky Survey DR7, we assemble a sample of 87 type 2 AGNs with double-peaked [O III] profiles. The nuclear obscuration in the type 2 AGNs allows us to determine redshifts of host galaxies through stellar absorption lines. We typically find that one peak is redshifted and another is blueshifted relative to the host galaxy. We find a strong correlation between the ratios of the shifts and the double peak fluxes. The correlation can be naturally explained by the Keplerian relation predicted by models of co-rotating dual AGNs. The current sample statistically favors that most of the [O III] double-peaked sources are dual AGNs and disfavors other explanations, such as rotating disk and outflows. These dual AGNs have a separation distance at ~1 kpc scale, showing an intermediate phase of merging systems. The appearance of dual AGNs is about ~10–2, impacting on the current observational deficit of binary supermassive black holes with a probability of ~10–4 (Boroson & Lauer).
        
        
- Liu+2010 (SDSS): https://doi.org/10.1088/0004-637X/708/1/427 (Tables have been downloaded)
    - Title: TYPE 2 ACTIVE GALACTIC NUCLEI WITH DOUBLE-PEAKED [O III] LINES: NARROW-LINE REGION KINEMATICS OR MERGING SUPERMASSIVE BLACK HOLE PAIRS?
        - Abstract: We present a sample of 167 type 2 active galactic nuclei (AGNs) with double-peaked [O III] λλ4959,5007 narrow emission lines, selected from the Seventh Data Release of the Sloan Digital Sky Survey. The double-peaked profiles can be well modeled by two velocity components, blueshifted and redshifted from the systemic velocity. Half of these objects have a more prominent redshifted component. In cases where the Hβ emission line is strong, it also shows two velocity components whose line-of-sight (LOS) velocity offsets are consistent with those of [O III]. The relative LOS velocity offset between the two components is typically a few hundred km s-1, larger by a factor of ~1.5 than the line full width at half maximum of each component. The offset correlates with the host stellar velocity dispersion σ*. The host galaxies of this sample show systematically larger σ*, stellar masses, and concentrations, and older luminosity-weighted mean stellar ages than a regular type 2 AGN sample matched in redshift, [O III] λ5007 equivalent width, and luminosity; they show no significant difference in radio properties. These double-peaked features could be due to narrow-line region kinematics, or binary black holes. The statistical properties do not show strong preference for or against either scenario, and spatially resolved optical imaging, spectroscopy, radio or X-ray follow-up are needed to draw firm conclusions.
        
        
- Smith+2010 (SDSS): https://doi.org/10.1088/0004-637X/716/1/866 (Tables have been downloaded)
    - Title: A SEARCH FOR BINARY ACTIVE GALACTIC NUCLEI: DOUBLE-PEAKED [O III] AGNs IN THE SLOAN DIGITAL SKY SURVEY
        - Abstract: We present active galactic nuclei (AGNs) from the Sloan Digital Sky Survey (SDSS) having double-peaked profiles of [O III]λλ5007, 4959 and other narrow emission lines, motivated by the prospect of finding candidate binary AGNs. These objects were identified by means of a visual examination of 21,592 quasars at z < 0.7 in SDSS Data Release 7 (DR7). Of the spectra with adequate signal-to-noise, 148 spectra exhibit a double-peaked [O III] profile. Of these, 86 are Type 1 AGNs and 62 are Type 2 AGNs. Only two give the appearance of possibly being optically resolved double AGNs in the SDSS images, but many show close companions or signs of recent interaction. Radio-detected quasars are three times more likely to exhibit a double-peaked [O III] profile than quasars with no detected radio flux, suggesting a role for jet interactions in producing the double-peaked profiles. Of the 66 broad-line (Type 1) AGNs that are undetected in the FIRST survey, 0.9% show double-peaked [O III] profiles. We discuss statistical tests of the nature of the double-peaked objects. Further study is needed to determine which of them are binary AGNs rather than disturbed narrow line regions, and how many additional binaries may remain undetected because of insufficient line-of-sight velocity splitting. Previous studies indicate that 0.1% of SDSS quasars are spatially resolved binaries, with typical spacings of ~10-100 kpc. If a substantial fraction of the double-peaked objects are indeed binaries, then our results imply that binaries occur more frequently at smaller separations (<10 kpc). This suggests that simultaneous fueling of both black holes is more common as the binary orbit decays through these spacings.
    
    
- Ge+2012 (SDSS) : https://doi.org/10.1088/0067-0049/201/2/31 (Tables have been downloaded)
    - Title: DOUBLE-PEAKED NARROW EMISSION-LINE GALAXIES FROM THE SLOAN DIGITAL SKY SURVEY. I. SAMPLE AND BASIC PROPERTIES
        - Abstract: Recently, much attention has been paid to double-peaked narrow emission-line (NEL) galaxies, some of which are suggested to be related to merging galaxies. We make a systematic search to build the largest sample of these sources from Data Release 7 of the Sloan Digital Sky Survey (SDSS). With reasonable criteria for fluxes, FWHMs of the emission lines, and separations of the peaks, we select 3030 double-peaked NEL galaxies. In light of the existence of broad Balmer lines and the locations of the two components of double-peaked NELs distinguished by the Kauffmann et al. criteria in the Baldwin-Phillips-Terlevich diagram, we find that there are 81 Type I active galactic nuclei (AGNs), 837 double Type II AGNs (2-Type II), 708 galaxies with double star-forming components (2-SF), 400 with mixed star-forming and Type II AGN components (Type II + SF), and 1004 unknown-type objects. As a by-product, a sample of galaxies (12,582) with asymmetric or top-flat profiles of emission lines is established. After visually inspecting the SDSS images of the two samples, we find 54 galaxies with dual cores. The present samples can be used to study the dynamics of merging galaxies, the triggering mechanism of black hole activity, the hierarchical growth of galaxies, and the dynamics of narrow line regions driven by outflows and a rotating disk.
        
        
- Comerford+2013 (AGES Field): https://doi.org/10.1088/0004-637X/777/1/64 (Tables have been downloaded)
    - Title: Dual Supermassive Black Hole Candidates in the AGN and Galaxy Evolution Survey
        - Abstract: Dual supermassive black holes (SMBHs) with kiloparsec-scale separations in merger-remnant galaxies are informative tracers of galaxy evolution, but the avenue for identifying them in large numbers for such studies is not yet clear. One promising approach is to target spectroscopic signatures of systems where both SMBHs are fueled as dual active galactic nuclei (AGNs), or where one SMBH is fueled as an offset AGN. Dual AGNs may produce double-peaked narrow AGN emission lines, while offset AGNs may produce single-peaked narrow AGN emission lines with line-of-sight velocity offsets relative to the host galaxy. We search for such dual and offset systems among 173 Type 2 AGNs at z < 0.37 in the AGN and Galaxy Evolution Survey (AGES), and we find two double-peaked AGNs and five offset AGN candidates. When we compare these results to a similar search of the DEEP2 Galaxy Redshift Survey and match the two samples in color, absolute magnitude, and minimum velocity offset, we find that the fraction of AGNs that are dual SMBH candidates increases from z = 0.25 to z = 0.7 by a factor of ~6 (from 2/70 to 16/91, or $2.9^{+3.6}_{-1.9}\%$ to $18^{+5}_{-5}\%$). This may be associated with the rise in the galaxy merger fraction over the same cosmic time. As further evidence for a link with galaxy mergers, the AGES offset and dual AGN candidates are tentatively ~3 times more likely than the overall AGN population to reside in a host galaxy that has a companion galaxy (from 16/173 to 2/7, or $9^{+3}_{-2}\%$ to $29^{+26}_{-19}\%$). Follow-up observations of the seven offset and dual AGN candidates in AGES will definitively distinguish velocity offsets produced by dual SMBHs from those produced by narrow-line region kinematics, and will help sharpen our observational approach to detecting dual SMBHs.
        

- Barrows+2013 (SDSS): https://doi.org/10.1088/0004-637X/769/2/95 (Tables have been downloaded)
    - Title: IDENTIFICATION OF OUTFLOWS AND CANDIDATE DUAL ACTIVE GALACTIC NUCLEI IN SDSS QUASARS AT z = 0.8-1.6
        - Abstract: We present a sample of 131 quasars from the Sloan Digital Sky Survey at redshifts 0.8 < z < 1.6 with double peaks in either of the high-ionization narrow emission lines [Ne V] λ3426 or [Ne III] λ3869. These sources were selected with the intention of identifying high-redshift analogs of the z < 0.8 active galactic nuclei (AGNs) with double-peaked [O III] λ5007 lines, which might represent AGN outflows or dual AGNs. Lines of high ionization potential are believed to originate in the inner, highly photoionized portion of the narrow line region, and we exploit this assumption to investigate the possible kinematic origins of the double-peaked lines. For comparison, we measure the [Ne V] λ3426 and [Ne III] λ3869 double peaks in low-redshift (z < 0.8) [O III]-selected sources. We find that [Ne V] λ3426 and [Ne III] λ3869 show a correlation between line splitting and line width similar to that of [O III] λ5007 in other studies, and the velocity splittings are correlated with the quasar Eddington ratio. These results suggest an outflow origin for at least a subset of the double peaks, allowing us to study the high-ionization gas kinematics around quasars. However, we find that a non-negligible fraction of our sample show no evidence for an ionization stratification. For these sources, the outflow scenario is less compelling, leaving the dual AGN scenario as a viable possibility. Finally, we find that our sample shows an anti-correlation between the velocity-offset ratio and luminosity ratio of the components, which is a potential dynamical argument for the presence of dual AGNs. Therefore, this study serves as a first attempt at extending the selection of candidate dual AGNs to higher redshifts.


- Shi+2014 (LAMOST Survey DR1): https://doi.org/10.1088/1674-4527/14/10/003 *(TABLES NOT DOWNLOADED)*
    - Title: A search for double-peaked narrow emission line galaxies and AGNs in the LAMOST DR1
        - Abstract: The Large Sky Area Multi-Object Fiber Spectroscopic Telescope (LAMOST) has released more than two million spectra, which provide the opportunity to search for double-peaked narrow emission line (NEL) galaxies and active galactic nuclei (AGNs). The double-peaked narrow-line profiles can be well modeled by two velocity components, respectively blueshifted and redshifted with respect to the systemic recession velocity. This paper presents 20 double-peaked NEL galaxies and AGNs found from LAMOST DR1 using a search method based on a multi-Gaussian fit of the narrow emission lines. Among them, ten have already been published by other authors, either listed as genuine double-peaked NEL objects or as asymmetric NEL objects, and the remaining ten are original discoveries. We discuss some possible origins for the double-peaked narrow-line features, such as interaction between jet and narrow line regions, interaction with companion galaxies, and black hole binaries. Spatially resolved optical imaging and/or follow-up observations in other spectral bands are needed to further discuss the physical mechanisms at work.


- Wang+2019 (LAMOST Survey DR4): https://doi.org/10.1093/mnras/sty2818 *(TABLES NOT DOWNLOADED)*
    - Title: Double-peaked narrow emission-line galaxies in LAMOST survey
        - Abstract: We outline a full-scale search for galaxies exhibiting double-peaked profiles of prominent narrow emission lines, motivated by the prospect of finding objects related to merging galaxies, and even dual active galactic nuclei candidates as by-product, from the Large Sky Area Multi-object Fiber Spectroscopic Telescope (LAMOST) Data Release 4. We assemble a large sample of 325 candidates with double-peaked or strong asymmetric narrow emission lines, with 33 objects therein appearing optically resolved dual-cored structures, close companions, or signs of recent interaction on the Sloan Digital Sky Survey images. A candidate from LAMOST (J074810.95+281349.2) is also stressed here based on the kinematic and spatial decompositions of the double-peaked narrow emission line target, with analysis from the cross-referenced Mapping Nearby Galaxies at the Apache Point Observatory (MaNGA) survey datacube. MaNGA enables us to constrain the origin of double peaks for these sources, and with the IFU data we infer that the most promising origin of double-peaked profiles for LAMOST J074810.95+281349.2 is the ‘Rotation Dominated  + Disturbance’ structure.


---------

Systematic searches have been conducted in other fields as well. Here are some major studies (found within the citations of these past four studies):

- Benitez+2013 (intermediate, double peaked AGN in SDSS) https://ui.adsabs.harvard.edu/link_gateway/2013ApJ...763...36B/doi:10.1088/0004-637X/763/1/36

- Shen+2013 (looked at both types of velocity offset selection for SDSS) https://ui.adsabs.harvard.edu/link_gateway/2013ApJ...775...49S/doi:10.1088/0004-637X/775/1/49


Looking ahead... for velocity offsets: Ju+2013 https://ui.adsabs.harvard.edu/link_gateway/2013ApJ...777...44J/doi:10.1088/0004-637X/777/1/44



Studies about double peaked lines and dual AGNs that I need to check into (which are derivatives of the above surveys...)
---------

Heckman+1981
Zakamska et al. 2003
McGurk+2015
Liu+2014
Comerford+2015
Comerford+2013
Foord+2020
Greene+2011 (for J0841 reference)
Basically every paper coming out of Comerford's group...



Papers on binary and double quasars from 1970-2000 (approximately), with some lens papers as well
---------

- Stockton+1972 (Ton 155 and Ton 156) --> (Reviewed 24 August 2020)
    - https://ui.adsabs.harvard.edu/abs/1972NPhS..238R..37S/abstract
    - Projected quasar pair. Project separation within about 35'' but disconcordant redshifts, and therefore are unlikely to be physically associated. 

- Hazard+1973 (4C11.50, aka 1548+115a+b) --> Reviewed 24 August 2020)
    - https://ui.adsabs.harvard.edu/abs/1973Natur.246..205H/abstract
    - Flagged a pair of blue stellar objects near the detection of 4C11.50. This object was follow-up in a parallel work by Wambler+ below.

- Wampler+1973 (1548+115a+b) --> (Reviewed 24 August 2020)
    - https://ui.adsabs.harvard.edu/abs/1973Natur.246..203W/abstract
    - Quasars are at a projected separation of ~5'', however lie at significanrly different redshifts. 

- Stannard+1974 (Follow-up on 4C11.50) --> (Reviewed 24 August 2020)
    - https://ui.adsabs.harvard.edu/abs/1974Natur.247..527S/abstract
    - Stannard+1974 followed-up on 4C11.50 to look for physical association of pair in the radio and finds no association.
   
   
- Wills+1974
    - https://ui.adsabs.harvard.edu/abs/1974ApJ...190..271W/abstract
    - There is some remarks about double radio sources reported by Cotton+1975, but no further information is given. No clear indication that these are distinctly different sources, or if they are, if they are at the same redshift. Just keep in mind for DR2..
    
- Browne+1975 
   - https://ui.adsabs.harvard.edu/abs/1975MNRAS.173P..87B/abstract
   - Doesn't seems to be relevant. Discusses one of the 'pairs' from Peterson (I think). But none of those were true physical pairs, so this is not relevant for us. 
   
- Bahcall 1974 --> Reviewed 24 August 2020
    - Makes only statistical arguments. Not need to include in catalog.
    
- Burbidge+1974 --> Reviewed 24 August 2020
    - Only makes statistical arguments, no need to include in catalog. 

- Gott+1974 (1548+115a,b) --> (reviewed 24 August 2020)
    - https://ui.adsabs.harvard.edu/abs/1974ApJ...190L.105G/abstract
    - Another interpretation of the pair of QSOs associated with 4C11.50: the foreground quasar is lensing the background quasar. Thus this is a lens candidate.
    
- Barnothy+1975 () --> Reviewed 24 August 2020
    - https://ui.adsabs.harvard.edu/abs/1975ApJ...198L..61B/abstract
    - Follow-up comment on Gott+1974's work, which proposed that the QSOs associated with 4C11.50 was a lens situation.

- Grewing+1975 
    - Looked at QSO/galaxy pairs. No need to include in catalog.
    
- Wall+1975
    - Discussed briefly some preliminary results but does not include object names or more details. No need to include in catalog. 
    
- Peterson+1976 (2 Pairs of QSOs at differetn redshifts) --> Reviewed 24 August 2020
    - https://ui.adsabs.harvard.edu/abs/1976ApJ...207L...5P/abstract
    - Spectroscopically confirms nature of QSOs selected from previous works, and in particular confirms the existence of two true QSO pairs. There were other pairs (I believe 9 other pairs) selected in previous works... Peterson only followed-up these two. But these are not at the same redshifts, and therefore will not be included! (At least in DR1).
    
- Rowan-Robinson+1976
    - Appears to only discuss observational evidence and known QSO pairs from Stockton+1972 and Wampler+1973. No need to include in catalog.
    
- Bolton+1976 (3 Pairs of QSOs at different redshifts) --> (Reviewed 24 August 2020)
    - https://ui.adsabs.harvard.edu/abs/1976ApJ...210L...1B/abstract
    - Follow-up observations of QSO pairs identified in Parkes 2700 MHz survey (i.e. pairs of UV excess objects detected within close proximity of radio sources. 
        - Overlap with Peterson+1976 and Browne+1975.
        - Initial results: evidence for 4 or maybe 5 pairs of QSOs in the sample of 6 that they've investigated so far (out of 8), but only three systems have redshifts. 
        - I am not including these systems as some either do not have redshifts, and those with redshifts are clearly not cases of binary quasars. Simply projections.

       
- Callahan+1976 appears irrelevant for our purposes.
    - No need to include.

- Moles+1978 discusses the 6 QSO pairs from Stockton+, Wampler+, and Bolton+, but does not include any new info.
    - No need to include.
       
- Sramek+1978
    - https://ui.adsabs.harvard.edu/abs/1978ApJ...221..468S/abstract
    - I don't think this paper is relevant. It mentions 'pairs' in the figures but not in the main text.
       
- Walsh+1979 (0957+561 A, B)
    - This, based at least on a quick skimming of the literature titles, is a gravitational lens. Will not be discussing this in DR1 of catalog. But will come back to it.
    
- Weedman+1980 does not appear to be relevant for us.

********
- Oort+1981 
    - appears very relevant and supposedly includes pairs of quasars with nearly identical redshifts... need to find a better copy than the copy I found on ADS.
    - The pairs in this are separated by 1 Mpc or more. Therefore no need to include in our catalog.
*********    
    
- Komberg+ looks like they had a sample of QSO pairs...? and they agreed with Oort+1981... but they provide no sample or sample of objects, so we're not including this in our catalog. 


- Weymann+1980 (Triple image QSO: PG1115+08)   --> To be included in DR2 
    - https://ui.adsabs.harvard.edu/abs/1980Natur.285..641W/abstract
    - 
    

- Margon+1981 (Nature: A companion quasar to 3C345) 
    - https://ui.adsabs.harvard.edu/abs/1981Natur.290..480M/abstract
    - Might be relevant to DR2 but not DR1. Separation is ~8' at z=0.5   
    
- Weedman+1982 (a third gravitational lens: 2345+007A,B) --> Reviewed 24 August 2020
    - https://ui.adsabs.harvard.edu/abs/1982ApJ...255L...5W/abstract
    - Intepreted as a gravitational lens, but other papers call into question whether this ia a true lens or a binary...

- Wright+1982 (Follow-up spectroscopy of PKS 0254-334)
    - https://ui.adsabs.harvard.edu/abs/1982MNRAS.199...81W/abstract
    - Just follow-up spectroscopic observations. Don't appear to contribute much more here...
    
    
- Steiner+1982 (Two associated associated with two different galaxy clusters)
    - https://ui.adsabs.harvard.edu/abs/1982ApJ...259..482S/abstract
    - Does not appear relevant to the study at hand... Not including.
    
********    
- Kapahi+1982 ('double quasar sample compiled from the literature...?)
    - https://ui.adsabs.harvard.edu/abs/1982JApA....3..465K/abstract
    - Claims to have pulled all 'classical double QSOs' or 'double qsos' from various other articles.... but I don't think these are actually 'double', other than perhaps two hot spots? I'm not sure.... will have to look into this soon... I think these are single quasars with double radio components? I'm going to exclude this paper from DR1.
*********    
    
- Gaskell+1983 (double-peaked quasars as binary smbhs)
    - https://ui.adsabs.harvard.edu/abs/1983LIACo..24..473G/abstract
    - Interesting paper to read, but does not provide any new binaries to our search. May want to cite though?

********
- Ford+1983 (follow-up to Arp's work, I think)
    - https://ui.adsabs.harvard.edu/abs/1983ApJ...266..451F/abstract
    - It looks like they rule out Arp's earlier work and find that the quasars are not at similar redshifts. Therefore I can exclude this paper and the paper they reference by Arp in 1980.
********

- Shaver+1983 (Just keep in mind for DR2)
    - https://ui.adsabs.harvard.edu/abs/1983Natur.303..155S/abstract
    - need to review this paper... but I don't think the pairs of QSOs listed here are 'close' by any means...
    
    
- Subramanian+1984 (lens intepretation of Q2345+007A,B) --> Reviewed 24 August 2020
    - https://ui.adsabs.harvard.edu/abs/1984ApJ...276..440S/abstract
    - Analysis of lens scenario for Q2345+007A,B, which was identified by Weedman+1982. Double lens scenario appears to be the best fit? Argues in favor of a lens interpretation. 
    
    
- Burke+1984 (Discusses 0957+561, 1115+080,2345+0007, and 2016+112) --> Reviewed 25 August 2020
    - https://ui.adsabs.harvard.edu/abs/1984ComAp..10...75B/abstract
        - offers a good review of the four proposed gravitational lenses to date (at this point in the literature).

********    
- Shaver+1984 (study on *close* quasar pairs)
    - http://articles.adsabs.harvard.edu/pdf/1984A%26A...136L...9S
    - Compiles list of quasar pairs and lenses from archives and literature. Looks like pure overlap for the true quasar pairs. Need to review further, perhaps later this evening.
********
    
- Djorgovski+1984 (Dual/lens 1635+267 A, B) --> Reviewed 25 August 2020
    - https://ui.adsabs.harvard.edu/abs/1984ApJ...282L...1D/abstract
    - QSO pair 1635 + 267 A, B confirmed to ahve similar redshifts (and were originally identified by Sramek and Weedman 1978 (but obviously not confirmed at that time). Djorgovski+ interpret this as a lens system, but later studies (Kochanek+, Mortlock+) question the interpretation and true nature.

- Weedman+1985 
    - https://ui.adsabs.harvard.edu/abs/1985ApJS...57..523W/abstract
    - The pairs discussed here are not too close in projected separation (closest is 25 arcsec), and they are at different redshifts. (The 25 arcsec sep. pair is actually at significantly different redshifts.)

**********
- Margon+1985 (some pairs of QSOs in an X-ray survey) 
    - https://ui.adsabs.harvard.edu/abs/1985ApJS...59...23M/abstract
    - I don't think the pairs in this sample are 'close'. I think they are on scales of a few arcmin from each other. Will need to double check on these... in fact, I'm not sure these 'pairs' are in fact true pairs. I think they are QSOs and foreground stars.
**********

- Djorgovski+1985 (PKS 1614+051 and companion [likely Sey2] galaxy) --> Reviewed 27 August 2020
    - https://ui.adsabs.harvard.edu/abs/1985ApJ...299L...1D/abstract
    - Using a novel Ly-alpha selection technique, Djorgovski+ find a companion to QSO PKS 1614+051 in CCD imaging, and follow-up spectroscopy confirms the redshift of the source and shows that the source is wjat would be expected of a higher redshift galaxy. The QSO and galaxy have very similar redshifts (~3.20) and are separated by ~6.55 arcsec. The second galaxy is likely a Seyfert 2 type system, meaning this is a QSO-AGN system. The QSO was detected in the radio but not the galaxy companion (which is also stated again in Djorgovski+1987).
        --> These may not be as close as I thought. Redshifts are 3.209 and 3.125...

- Owen+1985 (3C 75 Dual AGN) --> Reviewed 5 August 2020 / 21 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/1985ApJ...294L..85O/abstract
    - Radio imaging reveals that 3C 75 contains two radio loud sources, with are composed of point sources coincident with the galaxy nuclei and each with their own jets. The projected separation is only ~7kpc and with a ~330 km s-1 velocity difference, suggesting these are a pair fo interacting systems, making it a dual AGN.

- Turner+1986, Phinney+1986, and Shaver+1986 discuss the wide separation pair 1146+111B,C  
    - This will not make it into DR1, as it's separation is over 2 arcminutes
    - Unlikely to make into DR2, to be honest.
    
- Robertson+1986 (projected set of three quasars: 
    - https://ui.adsabs.harvard.edu/abs/1986MNRAS.219..403R/abstract
    - Project set with ~4' sep. Not likely to be included in DR1 or 2.
    
- Jakobsen+1986 (wide separation pair)
    - Incredibly large projected separation. likely not relevant to DR1 or 2.

- Bahcall+1986 (discussion of the three known close QSO pairs; pair vs. lens) --> Reviewed 245 August 2020
    - https://ui.adsabs.harvard.edu/abs/1986Natur.323..515B/abstract
    - Discussed briefly the three known quasar pairs at the time. Comments on the fact that the evidence is insufficient to state whether each of the three is a lens or a true pair.

- Peterson+1987 (Proposed binary in NGC 5548) --> Reviewed 26 August 2020
    - https://ui.adsabs.harvard.edu/abs/1987ApJ...312L...1P/abstract
    - In interpreting the two distinct emitting regions in NGC 5583 which are at different radial velocities, Peterson+1987 sugges this could be epxlained plausibly by a pair of SMBHs / pair of active nuclei. I have no idea if this is ruled out later in the literature, so I'll need to keep my eye on it...
    
- Djorgovski+1987 (QSO-AGN companion system PKS 1614+051) --> Reviewed 27 August 2020
    - https://ui.adsabs.harvard.edu/abs/1987AJ.....93.1318D/abstract
    - Follow-up imaging and spectroscopy of PKS 1614+051. Authors find a faint continuum for the companion ("A"), which argues against A being just a cloud of Ly-alpha emitting case ionized by the QSO. VLA imaging again shows the QSO is radio loud while A is radio quiet. It still seems abit uncertain as to whether the companion is indeeda true Seyfert 2 or  a star formation dominated system. It might be a marginal Seyfert 2, but regardless the evidence suggests it is a true galaxy companion. Djorgovski+ comment on the findings of Hu and Cowie+1987, who argue based on shallower r-band imaging that the companion is just a cloud of ionized gas, but Djorgovski point out that the higher S/N in this work and spectroscopy more favors the galaxy interpretation.
    
- Hu+1987 (Alternative interpreation on PKS 1614+051) --> Reviewed 27 August 2020
    - https://ui.adsabs.harvard.edu/abs/1987ApJ...317L...7H/abstract
    - Interpret this possible QSO-AGN system as instead a QSO with a large gas cloud nearby ionized by the QSO, and could possibly be a cooling flow rather than due to tidal interactions. Cites some work by Stockton on objects akin to this phenomenon... should check that out.
    
- Hewitt+1987 (0023+171 AB,C) --> Reviewd 1 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/1987ApJ...321..706H/abstract
    - Authors identified a triple radio source in VLA imaging: one double source (A+B) and a separate source located 5.9 arcsec away which is core only. Follow-up optical spectroscopy showed two quasars at identical redshifts with remarkably similar spectra. The optical sources AB_opt and C_opt are separated by 5.9 arcsec. Both AB and C are strong radio sources. The sources have similarly faint optical emission relative to their radio emission. It is argued that this is a likely lensed quasar system, but the binary interpretation cannot be ruled out, as the differences in radio morphology cannot be readily explained, the optical continuum and optical line flux ratios are different, and neither the optical nor ratio provide strong evidence for a galaxy or cluster cose enough to act as a lens. 

- Djorgovski+1987 (Binary: PKS 1145-071 A,B) --> Reviewed 25/26 August 2020
    - https://ui.adsabs.harvard.edu/abs/1987ApJ...321L..17D/abstract
    - While two other systems are known to be lens/binary candidates, PKS 1145-071 represents perhaps the first clear case of a true binary quasar system. Separation of ~4'' and same redshift determined for both systems. Optical spectra appear very similar with exception of strength of C IV, but A is radio loud while B is radio quiet... as the intensity difference in the radio is roughly ~500, whereas optical is only ~2.5. No variability detected on scales of a few months.
    
- Jakobsen+1988 (quasars in a supercluster?)
    - https://ui.adsabs.harvard.edu/abs/1988ApJ...326..710J/abstract    
    - Incredibly large angular separation. Definitely not to be included in DR1.
    
- Gaskell+1988 (more discussion on double-peaked emitters and smbh binaries)
    - https://ui.adsabs.harvard.edu/abs/1988LNP...307...61G/abstract
    - I cannot seem to get a copy of this from Springer without paying for it... leave for now. 

- Halpern+1988 (Test of hinary hypothesis in Arp102B) --> Reviewed 29 August 2020
    - https://ui.adsabs.harvard.edu/abs/1988Natur.331...46H/abstract
    - Place constraints on any possible binary existing in Arp 102B, and therefore I've included this system as a candidate binary, although I need to keep an eye on the literature to see if this object comes up again.
 
- Sillanpää+1988 (OJ 287) --> Reviewed 26 August 2020
    - https://ui.adsabs.harvard.edu/abs/1988ApJ...325..628S/abstract
    - Period outbursts observed in optical light curve every ~11.5 years (or ~9 years in the rest frame of the system) are proposed to arise from a pair of bound SMBHs with a separation of ~0.1 pc.
    
- Crampton+1988 (1343.4+2640: A Close Quasar Pair) --> Reviewed 26 August 2020
    - https://ui.adsabs.harvard.edu/abs/1988ApJ...330..184C/abstract
    - Two quasars separated by ~9.5'' first identified on a grens plate and then confirmed as quasars at nearly identical redshifts via optical spectroscopy. The quasars have substantially different emission line strengths and have substantially different absorption lines, making it unlikely that this pair is a lens and more likely to be a true binary quasar system. No foreground lensing system is identified in deep optical imaging, and no obvious structure in Ly-alpha around the two. While likely to be true binary quasar, the lensing hypothesis cannot be ruled out completely. 
    
    
- Mazzarella+1988 (Mrk 266) --> Skimmed on 26 August 2020
    - https://ui.adsabs.harvard.edu/abs/1988ApJ...333..168M/abstract
    - One of the earlier papers on this target, which discusses some possibility of both nuclei being active. Also cites Petrosian+1980 as being the first to state that the two nuclei are active (while others have instead stated one is active and one is a LINER, calling into question whether two AGN are present).... we'll more on this later.
    
    
- Meylan+1989 (UM 425: interpreted as a possible lens) --> Reviewed 26 August 2020
    - https://ui.adsabs.harvard.edu/abs/1989ApJ...338L...1M/abstract
    - Components A and B are consistent with being at the same redshift with a separation of ~6.5 arcsec and at a redshift of ~1.465. Optical spectra are very similar in shape (except B is slightly redder), and both have quasar spectra and the same emission lines with comparable or equal EW. Subtraction of the two spectra leaves residuals that are suggestive of an early-type galaxy at z~0.6, and thus could be the lensing system. Some evidence of a cluster along the line of sight at much lower redshift. Stands as a possible lens system, but the binary hypothesis cannot be ruled out at this point. (See papers by Mathur+2003 and Aldcroft+2003 later on...)
    
- Hewett+1989 (Q1429—008 A, B) --> Reviewed 1 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/1989ApJ...346L..61H/abstract
    - The two quasar images reside at identical redshifts and exhibit extremely similar optical spectra. No lensing body is found, but intervening Mg II absorption systems are observed at lower redshifts. The images reside at a redshift of z=2.076 with an angular separation of 5.14 arcsec. Hewett consider it a lens candidate but the bianry interpretation cannot be ruled out at this time. 
    
- Meylan+1990 (Likely binary PHL 1222)   --> Reviewed 26 August 2020  
    - https://ui.adsabs.harvard.edu/abs/1990Msngr..59...47M/abstract
    - Two quasars originally spotted in multi-color CCD imaging and confirmed as quasars spectroscopically with a separation of 3.3 arcsec and with nearly identical redshifts of ~1.91. The quasars exhibit substantially different spectral shapes (continuum of B is flat while rises sharply from blue to red), relative intensities of C III and C IV are reversed between the two. Spectral dissimilarities argue against lensing, and no foreground lens is discussed. Will need to keep an eye out for this in the literature later on.
    
    
- Graham+1990 (Arp 220, two active nuclei proposed) --> Reviewed 26 August 2020
    - https://ui.adsabs.harvard.edu/abs/1990ApJ...354L...5G/abstract
    - Proposes (for the first time?) that Arp 220 could contain two active SMBHs in the two detected nuclei.
    - But no concrete proof as of yet?...

- Miller+1990 (Argues against relativistic accretion disk) --> Skimmed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1990ApJ...361...98M/abstract
    - Need to probably review this again. I THINK they argue against both the binary hypothesis as well as the relativistic accretion disk hypothesis...
    
    
- Webb+1990 (Sinusoidal variability of 3C 120) --> Reviewed 4 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/1990AJ.....99...49W/abstract
    - Analysis of 61 years of optical observations of 3C 120 show that the optical lgiht curve can be separated into a linear component, sinusoidal component, and rapid, high-amplitude flares. While Webb brings up several mechanisms which might explain the observed phenomenon, such as accretion disk models, periodic feeding, or magnetoid models to explain the sinusoidal variability (double check this before you type this in your paper, Ryan...), they also bring up the Valtonen+89 model and suggest that it could also explain the observed sinusoidal variations. Caproni+2004b further examines this object. 

- Weir + Djorgovski+1991    (Double QSO Q2345+007) --> Reviewed 27 August 2020
    - https://ui.adsabs.harvard.edu/abs/1991AJ....101...66W/abstract
    - Find that the results from their new image restoration and recent spectroscopic results support binary scenario for this system.
    
- Djorgovski+1991 (Nice little overview of current close pairs) --> Reviewed 27 August 2020
    - https://ui.adsabs.harvard.edu/abs/1991ASPC...21..349D/abstract
    - No need to include in catalog. But includes a nice overview of the quasar pairs at different separations.
    
- Stockton+1991 (OX 169: double-peaked emission consistent with a binary SMBH) --> Reviewed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1991ApJ...371..525S/abstract
    - The observed broad Balmer lines are unusual; the H-beta is highly symmetric and there is a sharpness and depth of the apparent trough between the two peaks. The accretion disk model is ruled out as it cannot appropriately explain the variations in the data between 1983 and 1989, and previous explanations for this target or similar systems cannot be used to explain the observed phenomenon. Instead, they argue that the data most naturally can be explained by the existence of two independent BLR regions, which do not necessarily imply a SMBH (as there could be jets or bipolar outflows). That being said, they argue that a SMBH binary, where each SMBH has its own BLR, would be consistent with the data. 


*********
- Zheng+1991 (3C 390.3: binary hypothesis brought up and not ruled out) --> Reviewed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1991ApJ...381..418Z/abstract
    - I need to read back through this, but I believe double-peaked emission lines are observed, and the authors do bring up the possibility that these could be explained by a SMBH binary scenario. But I believe they said the data are also consistent with being due to an accretion disk as well. 
*********


- Duncan+1991 (Double QSO as a lens: Q2345+007) --> Reviewed 27 August 2020
    - https://ui.adsabs.harvard.edu/abs/1991ApJ...375L..41D/abstract
    - Based upon observed pattern of Ly-alpha forest absorption patterns, argues in favor of the lens hypothesis for this system.
    

- Steidel+1991 (Q2345+007 as a lens) --> Skimmed 27 August 2020
    - https://ui.adsabs.harvard.edu/abs/1991AJ....102.1610S/abstract
    - Long paper, but argues in favor of the lensing hypothesis for Q2345+007

- Magain+1992 (QSO pair G1208+1011, likely lens but cannot rule out binary) --> Reviewed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1992A%26A...253L..13M/abstract
    - Nature paper reports on discovery of multiply imaged quasar (2 images) at z=3.803. There is some evidence for absorbing systems in teh range 2.5<z<3.1, at which a lensing source could exist. Separation of the images based on ground based imaging is 0.45 arcsec. Initial imaging, follow-up imaging, and HST imaging all agree with conclusion that there is elgonation and multiple images. Cannot rule out binary hypothesis at this time, however.

- Hummel+1992 (1928+738) 
    - https://ui.adsabs.harvard.edu/abs/1992A%26A...257..489H/abstract
    -
    
    
    
    

- Maoz+1992 (HST imaging of QSO lens candidate Q1208+101l)  --> Reviewed 1 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/1992ApJ...386L...1M/abstract
    - HST observations of this binary / lens candidate reveal that this quasar is actually resolved into THREE separate point sources. While there is a non-negligible chance that one of two of these is a foreground star, the authors largely rule this scenario out. The observed image configuration cannot be reproduced with a simple lens and likely reqiures a more complex lensing geometry. The quasar is radio quiet. I will keep this in my DR1 list for now (As of 1 Sept. 2020) but it will likely be removed from DR1 as this imaging really seems to rule out the binary scenario... I'll keep any eye on the literature. 

- Maoz+1992 (HST imaging of lens candidate quasars) --> Skimmed 21 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/1992ApJ...394...51M/abstract
    - This paper reports that, whiel the HST images from the previous paper reported that the qausar was resolved into three separate images, but follow-up observations do not show this, thus indicating that teh previous thrid source was likely an artifact. Thus I will add this citation and keep this system as a bianry quasar candidate. 

- Roos+1993 (Binary in 1928+738?) --> Reviewed 4 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/1993ApJ...409..130R/abstract
    - This paper is relevant to DR1 but I'm finding it quite dense and difficut to pull information from. The authors note that Hummel+1992 observed 1928+738 over a period of 5 years and found that the visual motion of the jet in 1928+738 shows a sinusoidal pattern of motion and they attribute it to jet precession. Roos+1993 disagrees and argues that it is unlikely that the rapid period motion would be due to geodetic precession of the spin axis of the hole. Rather, they find that the motion of the jet is better characterized by the orbital motion of a SMBH binary with orbital radius 1e+16 cm (~3e-6 parsec). 
    
- Wisotzki+1993 (HE 1104 1805: gravitational lens or binary?) --> Checked 27 August 2020
    - https://ui.adsabs.harvard.edu/abs/1993A%26A...278L..15W/abstract
    - Glanced at this paper, which proposes either a binary or lens hypothesis, but a quick check with CASTLES suggests this is probably a lens, so I'm not going to worry about it. Quick skim of paper really suggests its a lens, too.

- Kochanek+1993 (Survey of lens candidates ) --> Skimmed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1993ApJ...417..438K/abstract
    - Large overview and analysis of lens candidates using a statistical method. Checking the conclusions, and it looks like the author argues in favor of binary hypothesis for UM 425. I can include this citation in the catalog for that target. They state PHL 1222 is a binary but based simply on literature, so no need to include there.

- Bonnet+1993 (Weak shear detected and analyzed in field of Q2345+007) --> Reviewed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1993A%26A...280L...7B/abstract
    - I found this paper sort of confusing. It seems that they found weak shearing in the field, and it places constraints on the allowed lensing masses/configurations in the fields, and they briefly mention that this could lend evidence to Q2345 being lensed, but it didn't seem definitive at all. I will includ the reference, but this paper doesn't seem to have removed any ambiguity about the situation.
    
- Fischer+1994 (Argues in favor of lens for Q2345+007) --> Reviewed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1994ApJ...431L..71F/abstract
    - Presents depper ground based imaging Q2345 and HST imaging. Rejects claims from some previous papers about splitting of image of Q2345B; this is not seen in HST and therefore is likely an artifact in teh previous papers. They identify a faint galaxy 1.5 arcsec from the B image, and find an excess of galaxies nearby the QSO. They develop a few different lens models that can reproduce the image positions and flux ratios but the problem is still underconstrained. Argues in favor of the lens hypothesis but further imaging, perhaps to search for a time delay in the images, is needed to conclusively decide the nature of this system.
    
- Mellier+1994 (Argues in favor of lens for Q2345+007) - > Reviewed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1994A%26A...289L..15M/abstract
    - Presents ultra deep CCD imaging of the field, and find an excess of faint objects at the center of the shear field determined in Bonnett+1993 as well as a small excess close to the QSO. No evidence for shearing of the faint excess of objects, which suggestes they are a foreground system. Thus they conclude that they've presented observational evidence for the lens responsible for teh weak shear and arc ocurrence in this field. The lens is argued to be complex, but the observational data obtained up to this point could be used to develop a good model for the multiply imaged QSO.
    
- Hewett+1994 (Binary/Lens Candidate Q1009-252) --> Reviewed 1 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/1994AJ....108.1534H/abstract
    - CCD imaging and spectroscopic observations reveal Q1009-252 to be a pair of quasars with a nearby foreground quasar. The similarity in their spectra, the existence of intervening Mg II absorption in both spectra, brightness of the components A and B, and that they are at identical redshifts suggests the pair may be a lensed quasar system. No lens is observed, though deeper imaging may be needed. However, the detection of [N III] in component B alone and not in A is at odds with the lens interpretation, and it cannot be readily explained by an extinction hypothesis. The equivalent widths of prominent emission lines also differ between the two spectra. The binary hypothesis cannot be ruled out, but a large amount of evidence can be reconciled with a combination of a lens scenario along with differential reddening.


- Crotts+1994 (Further evidence for Q1343+266A,B as a binary system) --> Skimmed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1994ApJ...437L..79C/abstract
    - Components A and B have remarkably different spectral features: A has strong Ne V, an associated narrow C IV absorber, and BAL features shortward of C IV emission. B has no BAL, narrow Ly-alpah than A, and weak Ne V emission. The quotient of the spectra show more irregularities than for any QSO lens. Thus based on spectral differences alone, these are highly likely to be distinct quasars.
    
    
- Dinshaw+1994 (Evidence for large ly-alpha clouds along line of sight of Q1343) --> Skimmed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1994ApJ...437L..87D/abstract
    - Spectroscopic observations taken of Q1343 show evidence for large Ly-alpah emitting clouds along the line of site. Observations agree with that of Crotts+1994. No direct discussion fo whether or not this is a binary or les; it is just taken to be a binary pair. Including here because the quoted redshifts differ slightly from that reported by Crampton+1988. 

- Dinshaw+1995 (Q 0107 - 025A and B wide separation pair)   --> Skimmed 27 August 2020
    - https://ui.adsabs.harvard.edu/abs/1995Natur.373..223D/abstract    
    - Might want to keep in mind for DR2

- Hjorth+1995 (New observations of Q1208; favor lens) --> Reviewed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1995A%26A...299..365H/abstract
    - While they cannot rule out the binary quasar hypothesis, and they do not find any evidence for a foregrounf lensing body, the authors argue that the time variation of the continuum intensity ratio and the difference in the continuum and emission line intensity ratios together suggest that component B is being microlensed

- Tinney+1995 (Lens/Binary Candidate QJ0240-343AB) --> Reviewed 1 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/1995MNRAS.277..609T/abstract
    - The system appears as tow quasar images which have effectively zero velocity difference between the two components (based on the broad lines) and the emission lines observed in the two spectra have similar strength and show storng similarities. There is a color difference but this could be explained via variability in the QSO continuum, the expected light travel time, and the delay between the data acquisition. The photometric and spectroscopic data strongly suggest this is a lens system, but the binary hypothesis cannot be ruled. Will keep my eye on the literature for this...
    
- Hagen+1996 (Likely binary HS 1216+5032) --> Reviewed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1996A%26A...308L..25H/abstract
    - As part of a follow-up spectroscopy campaign for the Hamburg Quasar Survey (HQS), two QSOs are identified at similar redshifts (A at 1.450 and B at 1.451) separated by 9.1 arcsec. The spectra show distinct differences, such as the broad C IV absorption line observed in B (B is possibly a BAL QSO). Deep imaging does not reveal any hints of an intervening lens mass. 

- Patnaik+1996 (Deep VLA imaging of Q2345+007) --> Reviewed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1996MNRAS.281L..17P/abstract
    - Deep VLA imaging revealed a radio source coincident with Q2345+007A but no source above the noise at the position of source B. The noise level is ~8 micro Janskies. This result is consistent with the lens hypothesis flux ratio between the two components, but it does not provide definitive proof and does not rule out the binary hypothesis.

- Vicente+1996 (Discussed radio observations of OJ 287) --> Skimmed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1996A&A...312..727V/abstract
    - I don't think we need to cite this paper in our work necessarily, although I'm keeping it in our paper list as it may have relevant papers that cite it.

- Dolan+1996 (HST UV observations of Q2345+007: favor lens hypothesis) --> Reviewed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1996Ap%26SS.240..295D/abstract
    - HST UV observations are said to be consistent with the scenario for a gravitationally lensed quasar.

- Pello+1996 (Observation of high redshift cluster near Q2345+007) --> Skimmed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1996A%26A...314...73P/abstract
    - Argues that the two images of Q2345+007 likely the result of a complex lens system involving different mass systems at different redshifts. They identify and characterize a rich cluster of galaxies at z=0.75, the center of which resides close to the center of the shear field. They also find a group of galaxies, rather than a cluster, at z=0.28.

- Taniguchi+1996 (theory paper) --> Skimmed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1996ApJ...469..581T/abstract
    - While this paper doesn't include ay new sources for our catalog (it is a theory work), it does bring up NGC 4736 (?) which was shown in Maoz+1996 to have two nuclear regions. No definitive evidence is provided though, regarding the possibility that the two nuclei are active... should keep an eye out for more info on that system though. 
    
- Puchnarewicz+1996 (Double-peaked Balmer emission in radio quiet AGN RX J1042+1212) --> Reviewed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1996MNRAS.283.1311P/abstract
    - X-ray selected radio quiet AGN which displays double-peaked H-alpha and H-beta emission lines. The accretion disk model is ruled out as an origin for the double peaks, but the data could be consistent with either the binary BH hypothesis or a double-jet model. The BBB for this source is very weak, and the optical to X-ray continuum may be dominated by a non-thermal component. The source is not detected in the radio or infrared. 
    
    
- Kochanek+1996 (SBS 0909+532: Binary or lens?)   --> Reviewed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1997ApJ...479..678K/abstract
    - Skimmed this and while Kochanek poses the question of whether this is a lens or true binary, CASTLES has this flagged confidently as a lens, and Mortlock does not include it in their Table of possible binaries. 
    - Keep this one in mind for DR2! A later paper (Lahar+2000) finds the lens responsible for this multiply imaged quasar.
    
- Valtonen+1997 (Brief discussion of models for OJ 287) --> Reviewed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1997ApJ...481L...5V/abstract
    - Provides a brief discussion of models and improvements of predictions for the outbursts in OJ 287 and how the latest models are accurate to a few days whereas the first models were accurate to roughly 1 year. Might be a good paper to cite unless some later papers are better. 
    
    
- Kundić+1997 (Has a small sample of quasar pairs) --> Skimmed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1997ApJ...482..631K/abstract
    - These pairs do not appear to be very close. It looks like they are slightly different redshifts (although maybe not too far away in redshift space?), but are on the scale of Mpc separations... might want to come back to this in DR2.
    
    
- Hawkins+1997 (QSO Lens/Binary Candidate Q2138-431) --> Reviewed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1997MNRAS.291..811H/abstract
    - The two quasars have essentially identical redshifts and remarkably similar colors and spectra. The study has failed to find a lensing system, although there is some evidence for some excess flux in K-band associated with the brighter of the two images, and therefore this raises the possibility that a potential lens could be sititng closer to the brighter of the two. However, since no lens has been detected but the data agree well with a lens intepretation, the authors suggest this may be a situation where a very red galaxy might be sitting along the line of sight, or given the mass to light ratio they calculated this might be a case of a dark lens.
    
- Hawkins+1997b (Statistical analysis of lensed/binary quasar candidates) --> Reviewed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1997A%26A...328L..25H/abstract
    - Authors use statistical argument to argue that the 6/8 close pair systems that are tentative lenses but for which no lens has been detected are more likely than not lensed quasars and are simply being lensed by dark lens systems. I could probably cite this in our paper though this does not add that much, and I'm wary of the small number statistics here...
    
- Small+1997 (Temporal changes in Q1634+267A,B and Q2345+007A,B) --> Skimmed 1 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/1997AJ....114.2254S/abstract
    - This paper examined spectra of ~30 quasars to look for changes in the spectra across two epochs in order to see if the spectral dissimilarities observed in the lens candidates Q1634+267A,B and Q2345+007A,B could be explained via time delays and variability. Indeed, the spectra from the two epochs for the 30 quasars are remarkably similar, though they do in about 2/3 cases exhibit changes in the equivalent widths or shapes of stronger lines on time scales of 1-1.5 years. Thus, it is possible that the spectral dissimilarities (i.e. EW differences, etc) in the lens candidates could be explained simply by temporal delays and variability.
    

- Eracleous+1998 (Accretion disk paper) --> 
    - https://ui.adsabs.harvard.edu/abs/1998AdSpR..21...33E/abstract
    - 
    
- Muñoz+1998 (MGC 2214+3550: A New Binary Quasar) --> Reviewed 31 August 2020
    - https://ui.adsabs.harvard.edu/abs/1998ApJ...492L...9M/abstract
    - VLA observations reveal jet-core morphology associated with A but no radio source associated with B is detected. The optical CCD imaging reveal two compact objects separated by 3.02 arcsec. Low resolution follow-up spectra revealed that not only was A a quasar, but so too was B (in their follow-up, they routinely obtained spectra of the nearest neighbor, they were not targeting B on purpose, so to say). The spectra are quite similar in shape, especially with the C III] and Mg II lines, however B's continuum rises slightly more rapidly than A's toward the blue. No lens system is observed in the CCD imaging, and no foreground absorption is observed in the spectra. The authors conclude that this is a true binary quasar. 

- Courbin+1998 (Lensed quasar HE1104-1805) --> Reviewed 1 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/1998A%26A...330...57C/abstract
    - This paper reports on the detection of the lens galaxy which is causing the multiple images observed in the multiply imaged quasar HE1104-1805. This will therefore not be included in DR1 but will be in DR2.
    
- Hewitt+1998 (Quasar pairs: 2153-2056 and 1148+0055) --> Reviewed 1 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/1998AJ....115..383H/abstract
    - The authors report on photometric and spectroscopic observations of two close quasar pairs found in the LBQS. The pair 1148+0055 have discordant redshifts (B is at a lower redshift) and therefore will be left out of DR1, but should be included in DR2. 
    - For the pair 2153-2056, the pair resides at roughly the same redshift of z=1.845 with a separation of 7.8 arcsec. The gravitational lens scenario cannot be ruled out, but it is argued that the lack of a galaxy over-density in the vinicity, the lack of any additional images (although they might be seen with this data quality), and that there is no evidence for a lens, as well as the soectral dissimilarities argues in favor of a true binary system. The spectrum of B shows BAL features from C IV absorption as well as the presence of Si IV absorption, which is not observed in A. Taking into account time delays from a lens model and intrinsic variability of the quasar source, one could conceivably account for the spectral differences. I'll consider this a binary / lens candidate for now...
    
- Dinshaw+1998 (Quasar pair LB 9605 and LB 9612) --> Skimmed 1 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/1998ApJ...494..567D/abstract
    - These two quasars are separated by 1.65 arcmin and are at slightly different redshifts (1.834 for LB 9605 and 1.898 for LB 9612) and therefore this pair will not included in DR1. Will probably be included in DR2. 
        

- Petitjean+1998 (Quasar pair Q1026-0045A,B) --> Skimmed 1 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/1998A%26A...334L..45P/abstract
    - This quasar pair resides at two different redshifts (1.438 and 1.520 for A and B, respectively) and the angular separation is 36 arcsec. This pair will not go into DR1 but might make it into DR2.
    
- Crotts+1998 (Examination of Ly-alpha forests around two quasar systems) --> Skimmed 1 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/1998ApJ...502...16C/abstract
    - This paper examines the quasar pair LB 9605 and LB 912 as well as the triplet system KP 76 KP 77 and KP 78. The quasars in the pair and in the triplet are at discordant redshifts and have very wide separations and therefore will not be included in DR1. Might be included in DR2.

- D'Odorico+1998 (QSO pair Q0307-195A,B + others) --> Skimmed 1 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/1998A%26A...339..678D/abstract
    - Only I think one of the small separation QSO pairs I've reviewed is in this, and this is about Ly-alpha measurements so I don't think we need to include this paper. The other quasar pairs/triplets are are all large separations and/or at discordant redshifts. The main pair in question (QSO pair Q0307-195A,B) are at very similar redshifts but separated by nearly an arcmin. No need to include in DR1, but might be good for DR2. 
    
- Kochanek+1999 (Overview of grav lenses vs. binaries and argues in favor of binaries) --> Reviewed 1 Sept. 2020    - https://ui.adsabs.harvard.edu/abs/1999ApJ...510..590K/abstract
   - 
    
- Giallongo+1999 (Proximity effect observed in Q1208+1011, argues for lens) --> Skimmed 1 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/1999ApJ...510..605G/abstract
    - The authors present new evidence based on the weakness of the 'proximity effect' mesasured in the high-resolution Lyman absorption spectrum of the QSO. The proximity effect consists of a reduction of the Ly-alpha line density in the region near the QSO emission redshift caused by the increase of the ionizing flux by the QSO. It depends on the competing ionization level produced by the diffuse UV background and the flux by the nearby QSO. They argue that the large number of saturated absorption lines in teh close vicinity of the quasar suggests that the ionizaton produced by the quasar is lower than for other QSOs of comparable apparent magnitude, and this would be expected if the system was gravitationally lensed. 
    
    
- Boyle+1999 (Quasar clustering study on AAT 2dF QSO survey; find one quasar pair/lens) --> Reviewed 1 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/1999RSPTA.357..185B/abstract
    - Follow-up spectroscopy of all catalogue pairs with angualr separations of 20 arcsec or less were being performs, and at the time of this publication 25% of the pairs had been observed, and this yielded onyl a single quasar pair/lens candidate: Q0101.8-3012A,B. The pair is separated by 17 arcsec on the sky and lies at a redshift of z=0.89. No other information is given; I'll keep an eye out for this in the literature and consider it a binary/lens candidate for now. 
    
- Brotherton+1999 (Dual Quasar: FIRST J164311.3+315618) --> Reviewed 4 August / 2 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/1999ApJ...514L..61B/abstract
    - Were observing targets as part of the FIRST survey and noticed in the acquisition image that there were two sources separated by 2.3 arcsec. They both sit at a redshift of 0.586; the broad Mg II absorption features are similar but the broad Balmer lines and narrow line spectra differ. B alson has a redder continuum and shows high-order Balmer Ca II H and K absorption lines. A is associated with a compact steep spectrum source (CSS) at 3.6cm in the VLA imaging, while B is radio quiet and associated with only a marginally detected source. A is a quasar and B is a Seyfert 1. The fact that the system is a radio loud/ radio quiet pair, as well as the fact that their optical colors and spectra differ significantly and there is no sign of lensing galaxy in the K' image argues strongly that this is a true AGN/quasar pair. The difference in colors appears to be consistent with stellar contributions from the host galaxy of B, which appears to be consistent with a recent starburst period.
    
- Fynbo+1999 (PHL 1222) --> Skimmed 1 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/1999MNRAS.305..849F/abstract
    - This paper looks at the damped Ly-alpha absorbers near the quasar pair PHL 1222. The paper does not discuss a binary vs.lens question, and is more interested in the Ly-alpha emission. No need to include this paper in DR1 study.

- Monier+1999 (Study on a cluster of five quasars acros several arcmin) --> Skimmed 1 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/1999ApJ...522..627M/abstract
    - This study is not relevent to DR1. Might want to keep in mind for DR2.
    
- Peng+1999 (HST observations of quasar binary/lens Q1634+267A,B) --> Reviewed 1 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/1999ApJ...524..572P/abstract
    - The authors present new HST imaging of the quasar pair Q1634+267A,B. The imaging confirms that there is no lensing galaxy, group, or cluster of galaxies near the quasar pair, and in fact they lack a candidate lens galaxy within 100 h^-1_65 kpc for producing teh observed Mg II absorption feature in spectrum A. The Mg II feature is observed in A but not in B (although I think previous papers concluded that). At this point, one cannot say conclusively if this system is a lens or a binary. The striking spectral similarity remains a mystery, and no evidence exists for a lensing body. They also obtained VLA imaging of Q1634+267, Q1120+0195, and LBQS 1429-008, but they do not detect any sources at 3.6cm.
 
- Mortlock+1999 () --> 
    - https://ui.adsabs.harvard.edu/abs/1999MNRAS.309..836M/abstract
    - 
    
- Porcas+2000 (quasar projected pair 1038+528 A and B) --> Checked 2 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2000AdSpR..26..673P/abstract
    - While this pair is separated by 33 arcsec, they are different redshifts (0.678 and 2.296 ). Keep in mind for DR2. 
    
- Rioja+2000 (quasar projected pair 1038+528 A and B) --> Checked 2 Sept. 2020 
    - https://ui.adsabs.harvard.edu/abs/2000A%26A...355..552R/abstract
    - On the same pair as Porcas+2000, which lie at different redshifts. Keep in mind for DR2.
    
- Morgan+2000 (CTQ 839, likely binary quasar) --> Reviewed 2 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2000AJ....119.1083M/abstract
    - CTQ 839 appears as two objects in all optical imaging filters, but was originally identified as a high redshift quasar in CTS. The optical flux ratio between components A and B appears to be wavelength dependent, changing between the R, B, and I bands. Stacked frames reveal no sign of a third component, which would be indicative of a lens galaxy, down to a magnitude of R=22.5. H band imaging yields similar results: the two components are resolved, the flux ratios differ significantly (1.5x larger) than the R and I band flux ratios, and no evidence for a third component is seen, despite the fact that any lens brighter than H=17.4 would have been detected. The quotient of their optical spectra reveal strong signs evidence for differences between the spectra in terms of equivalent widths and related emission features. VLA imaging, shallow and deeper, do not reveal any sign of radio emission coincident with the brighter of the two components (with 5 arcsec), and therefore this is classified as a radio-quiet/radio-quiet system. Einstein de-Sitter cosmology places strong constraints on teh lens hypothesis, while lambda-cosmology cannot for or against the lens hypthesis based on the detection limits here. The authors argue based on the spectral dissimilarities, lack of lens candidate, and broadband flux differences that this is likely not a lens but rather a true binary.

- Lopez+2000 (Lens/Binary candidate HS 1216+5032 AB) --> Reviewed 2 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2000A%26A...357...37L/abstract
    - This sytem was originally identifiued by Hagan+1996, and in a study originally intended to examine the Ly-alpha forest, Lopez+2000 also briefly discuss the merits of the binary vs. lens hypothesis. There are several reasons to argue in favor fo the binary: A BAL is observed in B but not in component A; no extended or luminous object (i.e. a lens) is detected near or even close to the QSO pair down to a limit of m_R=22.5; a lack of trend for equivalent widths of intervening Ly-alpha lines common to both spectra to become similar at redshifts closers to z_e also argues for a pair. But, the gravitational lens hypothesis cannot be ruled out at this time

- Claeskens+2000 (Q1548+114 A & B and Q1148+0055 A & B) --> Skimmed 2 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2000A%26A...356..840C/abstract
    - Q1548 will not be in DR1 due to discordant redshifts. 1148 will also not be in DR1 due to discordant redshifts. These will be kept in mind for DR2 as these are projected pairs / lens candidates.

- Fynbo+2000 (Ly-alpha study of PHL 1222) --> Skimmed 2 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2000A%26A...358...88F/abstract
    - Doesn't really say anything about the double quasar system so I don't think we need to include this in DR1, maybe not even in DR2. 

- Gregg+2000 (Likely lens HE 0512-3329) --> Reviewed 2 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2000AJ....119.2535G/abstract
    - While the binary hypothesis cannot be ruled out completey at this time, there is a significant amount of evidence in favor of the lens hypothesis. The images are separated by 0.622 arcsec. After PSF subtracted, there is an excess of counts between the two images and closer to the brighter component, which the authors interpret as the nucleus of the foreground lens galaxy. There is also an excess below A and to the right of B. The photometry shows that A is redder than B, and the authors argue that this can be explained via diffential reddening along the line of sight or via microlensing by stars in teh intervening lens. I will probably include this in DR1 for now (as of 2 Sept 2020) but I will keep my eye out for more info on this target.
    
- Lehár+2000 (HST imaging of several grav lenses, including 1009-0252) --> Skimmed 2 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2000ApJ...536..584L/abstract
    - This group identified the lens responsible for the multiple images seen in J1009-0252, and therefore this images quasar system is being bunmped to DR2.
    
    
- Schneider+2000 (pair of high-z quasars) --> Reviewed 17 August 2020 / 2 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2000AJ....120.2183S/abstract
    - The quasar pair is 33.4 arcsec apart and reside at identical redshifts 4.25. During observation of one high z quasar candidate, the slit orentation by chance included a second object which turned out to be a second quasar at the approximately the same redshift. Given the large separation and different profiles of Ly-alpha emission line and the contrast in strengths of the Si + O feature between the two, the lens scenario is unlikely. Both are radio-quiet. Redshift difference is consistent with zero with an error of 0.01 in z. There is evidence to suggest that A undergoes significantly less Ly-alpha absorption than B. Despite the large separation, I will for the time being include this in DR1. 
    
- Muñoz+2000 (Lens or Binary Quasar in RX J0921+4529) --> Reviewed 19 August 2020 / 2 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2001ApJ...546..769M/abstract
    - Two quasars detected coincident with large X-ray detected cluster. Excess of galaxies in the field as demonstrated by R band imaging, which also reveal the two blue objects, the quasars. Spectra obtained for several nearby galaxies shows redshifts of ~0.32 whereas the quasars are at z~1.66 with separation ~6.9''. Redshift of putative lens galaxy observed between the two could not be measured, although it is observed in HST imaging and is a galaxy with an apparent spiral structure. After subtracting the quasars, there is a faint residual near component B (with a slight offset between B and the residual); while it is likely a faint cluster galaxy, they cannot rule out that it is the host galaxy of B, in which case this would be a binary system. However, they show that if this is the host galaxy, and it is a lensed system, they should have detected this feature in A very easily but they do not. Putative lensing spiral galaxy and cluster have photometric redshifts consistent with z~0.32. Conclude that this is likely a lensed quasar, but follow-up observations could solidify this conclusion. While this is more than likely a lens system, I will keep in DR1 for now... ooooo but Popović coming in clutch in 2010 to claim this is a true binary!
    
- Morgan (Lens/Binary Candidate FBQ 1633+3134) --> Reviewed 2 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2001AJ....121..611M/abstract
    - FBQ 1633+3134 comprised two quasara at z=1.52 with a separation of 0.663 arcsec. The broadband photometric fluxes between the two are very similar across B,V,R, and I, and the optical flux ratios are consistent with the flux ratio upper limit derived from the VLA observation. No lens candidate is observed in the optical imaging, and the authors conclude no lens brighter than I=23 mag resides between the two quasar images (and there is no residual seen between the two after PSF subtraction). Spectroscopic observations show that they are clearly quasars. An intervening foreground absorber is present at redshift z=0.685 as evident from strong Mg II and Fe absorption lines. The seeing was not good enough to separate the two sources, so the spectra used is a composite of the two. The observed image could be reproduced with a slightly underluminous spiral or elliptical lens, although the authors calculate that a late type spiral shoudl have been detected, while an early type elliptical could go unnoticed. They ultimately conclude that this is a good len candidate but cannot rule out the binary hypothesis. 
    
- Junkkarinen+2001 (Dual Quasar: LBQS 0103-2753) --> Reviewed 31 July 2020 / 2 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2001ApJ...549L.155J/abstract    
    - HST imaging of two quasars separated by 0.3 arcsec. HST STIS spectra show that the quasars have very different emission, absorption, and continuum characteristics. Component A shows strong BALs, and the spectra between A and B show differences in terms of emission line shapes, equivalent widths, and and intensity ratios. While the two components reside at redshifts z=0.858 (B) and z=0.834 (A), the redshift of A was derived from the C IV line which often shows blue shifts in quasar spectra, so it is possible that these are actually closer in reality. With a time delay of ~10 days, it is unlikely that variability or microlensing could account for the observed spectral differences, and differenctial reddening cannot account for the large spectral differences. Variability is observed between data of a few years apart showing A changing from a loiw-ionization BAL to a high-ionization BAL. The authors conclude this is a true binary quasar.
        - This object is later reported on by Shields+ who show that the quasars indeed lie at the same redshift based on IR imaging and that the redshift derived from the C IV line in comp. A is not reflective of the true redshift. 
    
- Zhdanov+2001 (quasar pairs with arcmin separations) --> Skimmed 2 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2001A%26A...372....1Z/abstract
    - This has quasar pairs with ~ arcmin separations. Not relevant to DR1 but might be in DR2. 
    
- Aracil+2002 (HST imaging of wide separation quasar pairs) --> Skimmed 2 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2002A%26A...391....1A/abstract
    - These quasar pairs are at discordant redshifts and have arcmin separations. Might be included in DR2 but not DR1. 
    
- Green+2002 (Chandra observation of Q2345+007A,B) --> Reviewed 19 August 2020 / 2 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2002ApJ...571..721G/abstract
    - Conclude that the X-ray data disfavors the lens scenario, despite the incredible similarity of the two QSO's optical spectra. No evidence is found for a single cluster lens or luminous lens galaxy. The closest clusters to this QSO pair are not massive enough at a distance of 220 h_20 kpc to produce the observed images. X-ray spectra of the two sources are claimed to be incompatible. Green+2002 conclude that this is likely a true binary quasar system.

- Gregg+2002 (Binary quasar: SDSS J233646.2-010732.6) --> Reviewed 3 August / 2 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2002ApJ...573L..85G/abstract
    - It was flagged as an extended quasar by SDSS and was therefore follow-up as a potential lens system. K' band NIR imaging revealed two components as well as three galaxies in near the pair. PSF subtraction revealed a slight excess, suggesting that the two components were not quite identical in shape, and extended diffuse light in K' persists near A, suggesting that perhaps the host galaxy (or galaxies of both) is being detected. Optical spectra were obtained for both, revealing that component A is a standard quasar with broad emission lines while B is a BAL quasar totally lacking any emission features. Component A resides at z=1.2853 and B resides at most likely z=1.2871. There is intervening Mg II and Fe II absorption in the spectrum of A. The short time delay in a lens scenario and the strikingly different spectra immediately rule out the lens interpretation. Component B is a member of the FeLoBAL class of quasars. 

- Gregg+2002 about a PKS target, SDSS0300???

- Hall about SDSS0300 and claims it is a binary??? (ApJS)


- Impey+2002 (Dual Quasar: LBQS 0015+0239) --> Reviewed 3 August 2020 / 2 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2002ApJ...574..623I/abstract
    - Original plate imaging showed only a single object, but follow-up CCD imaging under good seeing conditions reveal two sources at the position of this quasar, and it was flagged as a lens/binary candidate. LRIS observations of the pair reveal that component A is ~10 times brighter than B and reveals that both show strong Ly-alpha, S IV, and C IV emission, but while strong N V is detected in the wings of Ly-alpha in A, it is virtually absent in B. A resides at redshift z=2.4747 and B resides at z=2.4831. Cross correlation shows a velocity difference that is different from zero at the 4 sigma level. Though no imaging in good seeing was obtained to rule out the lens interpretation, the authors inspect their spatially resolved spectra to place constaints on the possibility of the lens interpretation. Spectroscopy rules out a simple lens interpreation where the lens is in between A and B, and in the case where the lens is very close or on top of B there is no evidence for any admixture of light from an expected early-type lens galaxy in the spectrum of B, which would be detected. Therefore, if a lens exists, it must be at least 30x underluminous. With this evidence, the authors concluce that this is likely a true binary. 
    
    
- Merritt+2002 () --> Read 24 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2002Sci...297.1310M/abstract
    - 

- Clements+2002 --> Skimmed 25 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2002ApJ...581..974C/abstract
    - 

- Komossa+2003 (Dual AGN in NGC 6240) --> Reviewed 31 July 2020 / 3 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2003ApJ...582L..15K/abstract
    - This is the best known dual AGN to date, displays two nuclear hard X-ray sources which both produce neutral Iron K-alpha line emission. They coincident with the optical nuclei. Not much else needs to be said lol. 

- Mathur+2003 (Proposed Dual quasar: UM 425A,B) --> (Reviewed 5 August 2020 / 3 Sept. 2020)
    - https://ui.adsabs.harvard.edu/abs/2003ApJ...589L...1M/abstract
    - Originally flagged by Meylan+1989 as a possible lens. Mathur+2003 propose it is a true pair. Aldcroft+2003 also published on this (see above). Claim that diffuse X-ray emission is in fact associated with a cluster around UM 425A, rather than a foreground cluster. The X-ray emission is estimated to be at least a factor 5 fainter than the minimum flux estimated for the foreground lens, and even if the observed X-ray emission was associated with a foreground cluster at z~0.5, the cluster is not massive enough to cause the observed lensed images of Um 425. In fact the geometry of the system argues that diffuse X-ray emission does not arise from a foreground cluster at all, but rather by a cluster at the redshifts of the quasars. The diffuse emission is observed closer to the primary component (brighter of the two) rather than dimmer one, which is the opposite of what would be expected  in a lens scenario... the cluster should be closer to the dimmer component. Furthermore, they find the X-ray spectra to be incompatible with one another.

- Sudou+2003 (SMBH Binary in 3C 66B?) --> Reviewed 4 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2003Sci...300.1263S/abstract
    - This study used VLBI phase referencing at radio frequencies, which allows positional measurements with an accuracy on the order of tens of micro-arcsec. They study the two radio galaxies 3C 66A and 3C 66B, where 3C 66B is the closer of the two at z=0.0215 and is the focus of the study while 3C 66A is the more distant one at z=0.44 and is used as a stationary position reference for the foreground object. 
    - Sudou+ claim to observe time varations in the position of the radio core at 2.3 and 8.4 GHz and find it can be fitted by an elliptical motion. The average fitted fore motion is ~1 year, and they rule out that this has anything to do with microlensing of the source by a star in our galaxy or our own motion around the sun (and hence they were seeing a parallax. They claim that precession motion of the jet naturally explains the observation that the major axis of the radio core orbit is longer at 2.3 than at 8.4 GHz, because the radio core at 2.3 GHz is expected reside at a much further distance from the central engine than at 8.4 GHz. Further argue that the observed core motion cannot be dominated simply by jet precession, and that orbital motion is required, and thus argue for the existence of a binary SMBH. In this picture, core motion at 2.3 GHz is dominated by jet precession and motion at 8.4 GHz is dominated by the orbital motion of the massive SMBH binary.

- Faure+2003 (Likely binary quasar: LBQS 1429-0053) --> (Reviewed 19 August 2020 / 3 Sept. 2020)
    - https://ui.adsabs.harvard.edu/abs/2003A%26A...405..415F/abstract
    - Identified by Hewitt+1989 as a lens system, Faure+2003 uses VLT and HST data to search for lensing cluster or galaxy. Find no evidence for a luminous lens galaxy between the two quasars. They find that the galaxy cluster possibly at z~1 (based on photometric redshifts) that is within 5" of the quasars could not alone explain the images of the quasars (and thus that cluster could not alone be a sufficient lens), and there is no evidence for a dark lens. Similarly, there is no evidence on large scales or small scales (1-5 arcsec) for a lens. Possible lensed host galaxy in component A which at it's luminosity would not be seen in B, but this may also be foreground blue galaxy or actually the host of A itself. Ruled as a likely true quasar pair rather than a lens.
    
- Heidt+2003 (Dual quasar: PKS 0537-441) --> (Reviewed 4 August 2020  / 21 Sept. 2020)
    - https://ui.adsabs.harvard.edu/abs/2003A%26A...406..565H/abstract
    - Argues against the lensing theory by other authors. Find extended OII emission, and evidence for a potential QSO in a companion galaxy, making this a binary QSO system (if true). Deep I-band imaging does not reveal any signs of a lensing galaxy. None of the four nearby galaxies are lensed background galaxies and actually quite close in redshift to PKS 0537-441. One of the companions has very broad Mg II absorption at z=0.885, whereas PKS 0537 resides at z=0.893. The close proximity of the the other (non-active) galaxies suggests a link between the environment and the level of activity in PKS. The binary interpretation, where component A1 is a mini FeLoBAL, is favored by the authors. We will call this a binary quasar candidate

 - Morgan+2003 (CTQ 327) --> Checked 4 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2003AJ....126..696M/abstract
    - This is very likely to be a true lens, as not only is it a double image but the lens galaxy is resolved in imaging after a PSF subtraction. There are apparently some spectral differences between components A and B. This will be in DR2. 
    
- Aldcroft+2003 (possible lens or binary: UM 425A,B) --> (Reviewed 18 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2003ApJ...592..710A/abstract
    - Aldcroft finds evidence for and against BOTH the binary and lens scenarios. In support of the lens scenario: similar redshifts, colors, and spectral type for both components; anamolous brightness of UM 425A and faintness of UM 425B, and there is some X-ray and H band evidence for an intervening galaxy group. In support of the binary: the sources show significantly different intrinsic absorption; any putative lensing galaxy group is ~6 times X-ray underluminous for the required lensing mass; significantly different UV emission-line profiles; emission line equivalent widths consistent with baldwin effect for apparent luminosity differences. Also, no evidence for intervening lens galaxy near the fainter counterpart UM 425B. In fact they find no evidence for a cluster or group which could be solely responsible for the lensing of UM 425. Also, just as Mathur+2003 did, Aldcroft+ found that the X-ray spectra of the two sources were significantly discrepant, in fact that the 3-sigma level. This finding is consistent with the evaluation of the hardness ratios of the sources, which differ at the 3-sigma level. 
    - They present similar findings to that of Mathur+2003 (see above).

- Murphy+2003 (Multi-epoch obs. of 1928+738 [claimed by Roos+1993 to be a binary]) --> Reviewed 4 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2003NewAR..47..633M/abstract
    - As with Roos+1993, I found this paper to be a bit enigmatic. I *think* they are arguing in favor of the binary hypothesis? They argue that the relativistic variable speed ejecta ballistic jet model is favored over the relativist helical jet model. 

- Morgan+2003 (SDSS J1650+4251: A New Gravitational Lens) --> Checked 
    - https://ui.adsabs.harvard.edu/abs/2003AJ....126.2145M/abstract
    - This is very likely to be a lens, as the quasar is doubly imaged and the lens galaxy is recovered in I band imaging. This will be in DR2.

- Djorgovski+2003 (Quasar pair associated w/ SDSS 0338+0021) --> (Reviewed 18 August 2020 / 3 Sept. 2020)
    - https://ui.adsabs.harvard.edu/abs/2003ApJ...596...67D/abstract
    - While this might be relevant for DR2, the separation for these z=5 quasars is over three arcmin, with a projected separation of several Mpc. This could be included in DR2 but I will be skipped DR1 for now.
      
- Stockton+2004 (Two AGN in 3C 294) --> Reviewed 3 August 2020 / 21 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2004ApJ...600..626S/abstract
    - Borne out of the observations and conclusions of Quirrenbach+2001, who used AO H and K' badn imaging to conclude that the stellar eastern component near 3C 294 was an active nucleus which was separated by the diffuse wstern component by ~1'', Stockton+ re-examined this system using new AO imaging, HST observations, and Chandra imaging. HST imaging was used to tie the guide star to the ICRS system. The new AO imaging (K') provided new revised coordinates for the eastern stellar component and is compared to the other wavelength observations. The radio core observed at 6cm lies very close to one of the bright ness peaks of the western component, but is certainly not associated with teh stellar eastern component. In fact it lies 0.89'' west and 0.04'' north of the stellar component. Chandra imaging of 3C 294 and the surrounding region reveal that the nuclear source is elongated; part of this is due to the off-axis position of the source, but there is in fact a secondary source that appears after PSF subtraction coincident with the eastern stellar source. Thus, this system consists of two X-ray sources within ~1'' of each other. 
    
    
- Ballo+2004 (Dual AGN candidate in Arp 299) --> Reviewed 16 July / 21 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2004ApJ...600..634B/abstract
    - The naming of the individual components in in this system is WRONG! Simbad makes a note of this; make sure to note this in the catalog. 
    - Preliminary BeppoSAX observations showed the system was CT. New Chandra imaging shows that the X-ray emission is extended, and three knots are clearly observed. When energies are limited to 6.3-6.9 keV, one knot disappears and the strongest X-ray sources are localized in two regions associated with the two merging galaxies. XMM shows sources of comparable brightness associated with the two galaxies. WHen observing the X-ray spectra, residuals present after fitting suggest the presence of lines between 6.3 and 7 keV. The Fe K-alpha line is observed in the spectrum of NGC 3690, and the spectral properties agree with this AGN being CT, strongly arguing that the CT AGN inferred from the earlier BeppoSAX observations resides in this galaxy (global fitting of the Arp 299 system confirmed earlier concllusion of the presence of a CT AGN). IC 694(?) is a bit more ambiguous, as rather extreme conditions could produce the observed emission line at 6.7 keV (the 6.7 keV line from highly ionized Fe can be produced by a high-temperature plasma); the data do not rule out a starburst origin, but like it was said before, the conditions would need to be rather extreme. Thus it remains a strong possibility that IC 694(?) also harbors an AGN which is fainter and not highly absorbed. Thus, this system quite possible consists of one CT AGN and one faint unobscured AGN. Unfortunately they do not list the source coordinates from Chandra (which would have been best), and while they list XMM coordinates, these look less accurate. I use the XMM coordinate for the Eastern source and the Simbad coordinates to NGC 3690 for the Western component. 

- Colpi+2004 (Stellar mass binary BH) --> Checked 4 Sept. 2020
    - This is not relevant to our catalog. 
      
- Maness+2004 (Tentative binary AGN: 0402+379) --> (Reviewed 5 August 2020 / 3/4 Sept. 2020)
    - https://ui.adsabs.harvard.edu/abs/2004ApJ...602..123M/abstract
    - First proposed binary case for CSO 0402+379      

- Caproni+2004a (3C 345: possible precession due to SMBH binary?) --> Reviewed 4 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2004ApJ...602..625C/abstract
    - VLBI observations had previously shown that 3C 345 has a parsec jet with superluminal components, which travel apparently along curved paths with variable velocities. An alternative interpretation of the heliocoidal appearance of the jet is due to precession. The authors invoke a model to explain properties of the jet, and they suggest that jet precession could be due to a SMBH binary in this system and they subsequently derive orbital parameters and masses for the possible binary. Of course this is very preliminary and would definitely need more follow-up... I will keep my eye out in the literature. 
    
    
- Liu+2004 
    - 
    - 

- Miller+2004 (Possible arcmin separated lenses/pairs of QSOs) --> Skimmed 23 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2004MNRAS.348..395M/abstract
    - In a search for lensed QSOs with ~arcmin separations, Miller+ take QSOs identified in the 2QZ survey, identified confirmed QSOs whose colors were consistent and whose redshifts were consistent (all within the uncertainties of the redshifts and colors). One candidate pair with a discrepant radio detections was removed. Of QSOs with confirmed redshifts, there were no systems with more than two. 38 pairs were selected, and while they say only 10 received follow-up spectroscopy, they list 11 pairs in Table 1 and 2. About half or a little less than half are more likely to be pairs, and the others are not definitively lenses and therefore all 11 pairs will be added to our catalog with appropriately labeled system types.
        - NOTE: The coordinates in Table 2 are listed in FK5, NOT ICRS. Will need to convert!
        - ONE TARGET OVERLAPS WITH THE LATER CATALOG BY HENNAWI: 2QZJ1435+0008A AND 2QZJ1435+0009B (IN HENNAWI'S CATALOG); THIS IS J1435+0008 AND J1435+0009 IN MILLER+.

- Fukugita+2004 (Suburu imaging and spectroscopy of quasar pair SDSS J1439-0034A,B) --> Reviewed 4 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2004ApJ...603L..65F/abstract
    - Authors obtained follow-up Suburu imaging and spextroscopy of the high-z quasar pair identified by Schneider+2000. They find significant differences in the optical spectra between the two quasars, particulasrly in the strength of the C IV line and the presence of strong absorption in the spectrum of B. They find redshifts of 4.228 for A and 4.243 for B and they find that the redshifts are marginally consistent with a single value. I will quote these redshifts in the catalog. Imaging results show that there is no recognizable clustering of galaxies near the quasar pair and that there is no apparent enhancement of galaxies in the field; the environment of the quasars differs little from a general field. The data presented strengthens the argument that this is a true pair and not a lensed pair. 

- Zhou+2004 (Binary AGN in SDSS J104807.74+005543.5) --> Reviewed 13 July / 4 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2004ApJ...604L..33Z/abstract
    - Selected based upon the observed doul-epeaked optical spectroscopic emission lines, Zhou report that these can viewed as two distinct NLRs, especially considering that VLBA imaging reveals two compact radio sources, suggestive of a binary system with a separation of 2.5 mas or 20 pc. 
    
- Caproni+2004b (Possible binary in 3C 120?) --> Reviewed 4 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2004MNRAS.349.1218C/abstract
    - The differences in apparent velocities of the different superluminal components in the milliarcsec jet can be explained by a precession model, and the authors consider a binary SMBH system as a potential cause for such precession. 
    
- Green+2004 (Likely true binary quasar: HS 1216+5032 A,B) --> (Reviewed 18 August 2020 / 4 Sept. 2020)
    - https://ui.adsabs.harvard.edu/abs/2004MNRAS.349.1261G/abstract
    - No lens system had been previously detected when this system was found in the Hamburg Quasar Survey (Hagen+1996), but UV HST data (Lopez, Hagen & Reimers 2000) found complex intervening C IV absorbers, suggesting this could be a dark lens. Green+2004 use optical and IR spectroscopy to show that there are significant differences in the profiles of the H-alpha emission (lack of narrow component in B), only B component shows strong BAL systems, the continuum shapes of the systems differ significantly (and this is bolstered by B-R colors), A is the brighter of the two in the optical and IR, but B is radio loud while A is radio quiet. Therefore, highly likely this is a true binary quasar pair.    
    --> I think Green+2004 puts this question to rest, and I see no citations to this other than the Veron-Cetty catalog.
    
    
- Klamer+2004 (Radio galaxies J0321−455N and J0321−455S [ akin to 3C 75])  --> Reviewed 4 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2004MNRAS.351..101K/abstract
    - This system consists of a pair of radio tailed galaxies which are part of a large galaxy cluster. The radio morphologies are strikingly similar to that observed in 3C 75, and are highly indicative of a dual AGN with separation of ~35-40 kpc.
    
- De Paolis+2004
    - This paper seems to only run some models on the proposed binary in 3C 66B and makes some predictions for future X-ray / gamma ray observations but does not contribute much. Will not be including.  
   
   
 - Guainazzi+2005 (Dual AGN: ESO509-IG066) --> Reviewed 29 July 2020 / 21 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2005A%26A...429L...9G/abstract
    - XMM imaging reveals two active nuclei in merging system ESO509-IG066. The western source is compton thin but the eastern source is unobscured. A significant contribution from shocked gas or supernovae is unlikely. In the optical, the western source is classified as a Seyfert whereas the eastern source is more favored to be an HII or LINER region. The observations here show definitively that there are two active nuclei in this merger. 
    

- Lobanov+2005 
    - 
    - 


- Gil-Merino+2005 --> Checked 21 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2005A&A...432...83G/abstract
    - This is on a quadruply lensed quasar. No need to include. 


- Green+2005 (Galaxy cluster observed near UM 425) --> Skimmed 21 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2005ApJ...630..142G/abstract
    - In a follow-up work, Green+ present VLT imaging and spectroscopy which shows that there is a foreground galaxy cluster but still no foreground primary lens is found. It is still unclear if this quasar pair is indeed a lens, and it would apparently have to be unusual one due to the large flux ratio and angular separation but lack of lens in optical and infrared imaging. The authors do not conclusively rule out the binary interpretation.

- Magain+2005 (HE0450–2958: QSO without an observable host) --> Read 22 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2005Natur.437..381M/abstract
    - The authors fail to find a host galaxy related to the QSO HE0450–2958. There is a companion galaxy that is clearly undergoing a strong gravitational interaction with presumably the QSO/host, despite no host being found. The QSO does lie at the edge of a 'blob,' a gas cloud that is being excited by the QSO itself. If a host galaxy is present around the QSO, it is at least 6 times fainter than should be expected for such a bright quasar. 
        - This discover is followed-up in a number of other papers, including Haenhelt+2006 and Letawe+2009 (and mabout 5-7 others).

- Hennawi+2006
    - 
    - 

   
- Pindor+2006 (Lens: SDSS J102111.02+491330.4, Binary: SDSS J1120+6711) --> (Reviewed 18 August 2020 / 4 Sept. 2020)
    - https://ui.adsabs.harvard.edu/abs/2006AJ....131...41P/abstract
    - SDSS J1021+4913: Likely lens; identical redshifts, spectral similarities, K' band imaging reveals evidence for a lens galaxy between the two images. System is well fit with a two component model for the quasar images and another component for the lens galaxy in the foreground. 
    - SDSS J1120+6711: likely binary; indistinguishable redshifts, but have significantly different SEDs, and optical and H' band imaging do not reveal any signs of a lens system (system is well fit with a two component model). The non-detection is inconsistent with a lens galaxy of typical luminosity. 
    - Table included at the end that includes a list of rejected lens candidates as well as two (??) potential binary quasars.   
   
  
- Haehnelt+2006 (Alternative explanation for HE0450–2958: a recoiling/slung-shot BH) --> Reviewed 23 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2006MNRAS.366L..22H/abstract
    - In an alternative interpretation than that provided by Magain+2005, Haehnelt+ suggest that the quasar HE0450–2958 could have been ejected from the companion ULIRG either via gravitational recoil after a BH merger event or could have been slungshot out of the ULIRG if the two progenitors both had hung-up harddened binaries at their cores. In the second scenario (hung-up binaries in the progenitors), one might expect to find one or two other QSOs on the other side of the ULIRG. We will likely call this a recoil candidate but will make a note that it is also a sling-shot candidate (though I think this is less likely). These authors quote the separation between the quasar and the companion ULIRG as ~1.5'' (between the quasar and from the center of the compaion).

- Basu+2006
    - https://ui.adsabs.harvard.edu/abs/2006AJ....131.1231B/abstract
    - I don't think this is a legitimate work. Essentially no one cites it except for the author, and the two papers that cite it: one completely negates the findings, and the other doesn't seem to talk at all about it? I don't know. Either way, not citing this. 
   
   
- Hudson+2006 (Chandra view of 3C 75, aka Abell 400) --> (Reviewed 18 August 2020 / 4 Sept. 2020)
    - https://ui.adsabs.harvard.edu/abs/2006A%26A...453..433H/abstract
    - X-ray cores of the dual radio core system are easily resolved, along with X-ray emission from cluster. X-ray sources are not highly absorbed, and this work proposes that the two AGNs 3C 75 are likely in a bound, merging system (or possible merger remnant) rather than an unbound (i.e. projected) system. Includes redshifts of the AGNs and X-ray coordinates of the AGNs.

- Rodriguez+2006 (Binary AGN: 0402+379) --> (Reviewed 5 August 2020 / 3/4 Sept. 2020)
    - https://ui.adsabs.harvard.edu/abs/2006ApJ...646...49R/abstract
    - Follow-up to Maness+2004, who first proposed the binary case. 


- Liu+2006
    - 
    - 

- Hennawi+2006 (Quasars Probing Quasars I) --> Checked 9 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2006ApJ...651...61H/abstract
    - While this paper includes some closely separated (projected separations) quasar pairs, they are all foreground-background pairs with redshift differences of delta z>0.01. Keep this in mind for DR2.


- Hudec+2006
    - 
    - 

- Basu+2006
    - https://ui.adsabs.harvard.edu/abs/2006JApA...27..381B/abstract
    - I don't think this is a legitimate paper, either. Again, essentially no one cites this work except for the author, who is a SINGLE author with no coauthors... The only other paper that appeared to cite this isn't even refereed...


- Inada+2007 (2 new gravitational lenses in SDSS) -->  Skimmed 21 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2007AJ....133..206I/abstract
    - Given the high likelihood that these are indeed lenses (one has a spectroscopically confirmed lens, the other's has a lens whos redshift was estimated based on photometry) based on the data and the ease of explaining these sources with a lens model, I will not be including these in our catalog. 
    
- Hennawi+2007 (Quasars probing quasars II) --> Checked 9 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2007ApJ...655..735H/abstract
    - Just like Paper I (Hennawi+2006a), these are foreground-background quasar pairs which redshift differences delta z > 0.01. Keep this in mind for DR2!

- Myers+2007
    - 
    - 
    
- Schröder+2007
    - 
    - 

- Gerke+2007 (Dual AGN: EGSD2 J142033.66+525917.5 in AEGIS/DEEP2) --> (Reviewed 13 July)
    - https://ui.adsabs.harvard.edu/abs/2007ApJ...660L..23G/abstract
    - During inspection of the 1D spectra to confirm redshifts, this system was found to have double-peaked optical spectroscopic emission lines. Examination of the 2D DEIMOS spectrum revealed that the system has two spatially offset emission regions, separated by 0.17'' on the sky. They derive a redshift of 0.709 for the system, find a velocity difference of ~630 km s-1 between the two components, with a projected separation fo ~1.2kpc. The optical spectroscopic emission line ratios places both emission regions within the Seyfert category. The authors also perform some SED fitting and examine Chandra data, and conclude that the data support the picture of the system hosting a dust obscured AGN. GALFIT results support the interpretation that this is a double nucleus system. The authors conclude that this is likely a dual AGN, but also bring up the accretion disk and outflow scenarios briefly. 

- Rieger+2007
    - 
    -

- Ellison+2007 (Quasar pair SDSS J1116+4118A/B, overlap with Hennawi+ catalog) --> Skimmed 21 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2007MNRAS.378..801E/abstract
    - While the majority of the science in this paper is not relevant to our catalog, the target in question, quasar pair SDSS J1116+4118A/B, is relevant. It also appears in Hennawi+2006. I will keep that entry in Hennawi+'s catalog, but I will substitute in the slightly improved redshifts from this work. I also need to add a citation for Sanchez-Alvaro+2007, as they concluded that this was a wide separation lens, but I will defer to Hennawi+ and Ellison+. Although, it looks like Sanchez-Alvaro was never actually published? There's only an arxiv listing? So perhaps it never made it through review? Eh, maybe that one doesn't need a citation. 

- Bonning+2007 (First every systematic search for recoiling BHs) --> Reviewed 17 August 2020 / 21 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2007ApJ...666L..13B/abstract
    - While Bonning+ searched for recoiling black holes based on velocity shifted QSO emission lines compared to the host galaxies, but their search does not return any convincing evidence for recoiling black holes. They highlight two objects which might merit some further investigation, but we will not include these in our catalog as there is little evidence presented to merit including as potential recoul candidates. 

- Sergeev+2007
    -
    -

- Cheung+2007 (Candidate X-shaped radio sources... possible binaries) --> Skimmed 22 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2007AJ....133.2097C/abstract
    - Cheung+ assembles a large sample (100) of candidate X-shaped radio sources via visual inspection of FIRST images. See paper about details on exact selection methodology. After identifying 100 candidate systems which appear to show X-shaped or winged rqdio structures, they cross-match with NED, SDSS, and DSS to obtain optical counterparts. The motivation for this work is to assemble a large sample of these types of objects to try and understand them as a population since so few (by their count ~17) have been thus far found. A proposed formation process for X-shaped radio structures is the rapid realignment of a radio jet following the merger of two SMBHs, hence we can consider these candidate SMBHs/recoiling black holes/merged black holes. Table 2 (the 100 candidates) will be added to our catalog, and FOR NOW we will include the Table 1 list but ONLY cite Cheung+ and will add a note for each to check Cheung+ Table 1 for the reference to the discovery paper. In future releases we will add in the appropriate references. 
    
    
- Myers+2008 
    - 
    - 


- Komossa+2008 (Recoiling SMBH in J092712.65+294344.0) --> (Reviewed on 14 July / 8 Sept. 2020)
    - https://ui.adsabs.harvard.edu/abs/2008ApJ...678L..81K/abstract
    - The system exhibits two sets of emission lines, one of very narrow emission lines, and a second set of broad Balmer and broad high-ionization forbidden lines blue shifts by 2650 km s-1 relative to the narrow emission lines. These systems are referred to as the "red" and "blue" systems respectively. More specifically, the blue system shows the usual broad Balmer and Mg II emission lines as well as unusually broad narrow emission lines, whereas the red system conists of very narrow emission lines. The line ratios endicate AGN-like excitation in both the red and blue systems. This system is interpreted as a recoiling SMBH system: the blue system consists of emission lines frmo gas bound to the recoiling SMBH and the red system of emission lines represent the NLR left behind and or the ISM surrounding the SMBH, but does not emit from material bound to the SMBH. In this picture, J0927 underwent a merger and the SMBH recoiled post-coalescence. 
    
    
- Bianchi+2008 (Dual AGN in Mrk 463) --> Reviewed 3 August 2020 / 21 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2008MNRAS.386..105B/abstract
    - Chandra X-ray imaging reveals two sources coincident with the galaxy nuclei in the double-nucleus merger Mrk 463. The spectra are typical of Compton-thin sources, with column densitys >70e+22 and >30e+22 cm**-2. The data, in concert with supporting HST data, unambiguously shown that mrk 463 hosts a dual AGN. In the X-rays the sources are separated by 3.83 arcsec, and we quote that as the angular separation in the catalog. Coordinates for the X-ray sources are not given, though. 
    
- Ivison+2008    
    - 
    -     
    
    
- Kirkman+2008 Investigating Ly-alpha study with QSO pairs --> Skimmed mid-September
    - https://ui.adsabs.harvard.edu/abs/2008MNRAS.391.1457K/abstract
    - The QSO pairs used in this study are drawn initially from the list of quasars in NED, selected to be g, BJ or V < 22, and selected so that the pairs have similar redshifts. The amount of time it would take to observe the pairs was also factored in, and so there eas a preference for observing closer pairs. ALl pairs with usable Ly-alpha from SDS were also included. With the exception of 9 systems, at the time of selection no were other QSOs known with 5 arcmin and V<22 of either QSO in each pair. The science of this paper is not relevant to the catalog were are putting together, but we will be taking the info from table 1 and using what we can (the pairs of QSOs). 


- Rodriguez+2009 (2 Sey 2 optical counterparts to Swift source IGR J20286+2544) --> Checked 9 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2009A%26A...494..417R/abstract
    - The two Sey 2 galaxies associated with the SWIFT source IGR J20286+2544 are situated close to one another (only about 92 arcsec or ~30 kpc at z=0.015/0.014), but they seem to be separated by ~5 Mpc in 'redshift'. Remving from DR1 but might be relevant to DR2. 

- Prochaska+2009 (Quasars probing quasars III) --> Checked 9 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2009ApJ...690.1558P/abstract
    - This focuses on one of the foreground-background quasar pairs from Paper II. This could be used in DR2, but is not relevant to DR1. 

- Tytler+2009 Investigating Metal absorption systems with QSO pairs --> Skimmed mid-September
    - https://ui.adsabs.harvard.edu/abs/2009MNRAS.392.1539T/abstract
    - The authors use spectra of 310 QSOs, which corresponds to 140 pairs and 10 triples. The sample is strongly biased towards closely separation pairs, but there are many with very large separations. The pairs are also biased towards having more siilar redshifts. The sample is also biased against pairs which have CIV BAL absorption at the absorption redshift. The science of the paper is not relevant to our catalog, but at least some of the targetted pairs are relevant to us.

- Morganti+2009 (HI observations of CSO 0402+379) --> Skimmed 9 Sept. 2020 / 21 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2009A%26A...496L...9M/abstract
    - They observe broad HI absorption in CSO 0402+379 consisting of two components separated by roughly 1100 km s-1 and centered on the systemic velocity of the host. The authors compare these observations to optical data, and they propose that, while the jet hypothesis of Maness+04 cannot be ruled out, the authors favor the interpretation that this is further evidence for the binary. Rodriguez+ (likely published in 2010) find this same broad absorption phenomenon.

- Foreman+2009  - Reviewed mid-September 2020   
    - ...
    -  This paper's sample is assembled from previously discovered QSO pairs, and therefore it has been removed from consideration. (At least for now...)
    
    
- Boroson+2009 (Candidate subparsec binary SDSS J153636.22+044127.0) --> Reviewed on on 14 July / 8 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2009Natur.458...53B/abstract
    - Two sets of broad emission lines are observed in the SDSS spectrum of this system, and is one of two that exhibit such features (the other being J0917 discovered by Komossa+2008), which one residing at z=0.3889 and one at z=0.3727 A set of narrow emission lines is observed in the spectrum, and this eission region lies at an intermediate redshift z=0.38783. I will quote the redshifts of the BLRs for the individual redshifts in the catalog. There are some peculiarities to the spectra: the absorption line system is atypical of low-redshift quasars, showing no evidence for stellar absorption features and lines are unresolved in the SDSS spectra. There is no narrow or foridden line emission in the b-system (the lower redshift system). The r system at high redshift shows broad balmer lines and the usual narrow lines in low z quasars, and the blue system exhibits broad balmer lines and broad Fe II emission, but as said before no narrow or forbidden emission lines.
    
- Shields+2009 (Comment on J0927 discovered by Komossa+08) 
    - https://ui.adsabs.harvard.edu/abs/2009ApJ...696.1367S/abstract
    - A follow-up study on the recoil candidate discovered by Komosso+2008. Shields+ find a third redshift system of weak, narrow emission lines at an intermediate redshift. They discuss many of the scenarios put forth to explain this system: recoil event, binary SMBHs, a superposition of QSOs. Overall it would appear that they argue for more data to be obtained and more analysis to be performed, but based upon their discussions, they make the statement that, since J0927 appears to reside in a substantial cluster and the appearance of a third velocity system close to the spectrum of J0927, the superposition hypothesis does deserve more attention. 
    
- Rodriguez+2009 (HI observations of 0402+379) --> Reviewed on 8 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2009ApJ...697...37R/abstract
    - They publish HI data for 0402+379, but the data only really focuses on HI clumps observed in the vicinity of one of the sources, and so this paper isn't super important for our catalog. I'll keep in the DR1 list, simply because I'll add it to the repository when we had a full list of papers. 
 
- Bogdanović+2009 (Follow-up on J0927 discovered by Komossa+08) --> Skimmed 8 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2009ApJ...697..288B/abstract
    - Boganovic+ suggests that the recoil candidate J0927 is in fact a binary SMBH system. They present their model, in which the primary (large) of the two SMBHs is quiescent or far less active than the secondary, and the secondary SMBH is the on that is actively accreting material. The secondary is responsible for the observed blue system broad lines. In this scenario, the red system emission lines (the narrow emission lines) are originating from the ISM of the host galaxy.


- Comerford+2009 (Dual AGNs in DEEP2)
    - https://ui.adsabs.harvard.edu/abs/2009ApJ...698..956C/abstract
    - They search the galaxies in the DEEP2  Galaxy Redshift Survey for systems which exhibhit double-peaked optical lines or offset optical emission lines. They identify 30 offset AGNs and two double-peaked AGNs, which are called dual AGNs but we will consider them dual AGN candidates until otherwise proven. The two dual AGN candidates based upon double-peaked OIII lines are EGSD2 J141550.8+520929 and EGSD2 J142033.6+525917 (the latter was also published by Gerke+2007). 
    
- Wrobel+2009 (Follow-up VLA imaging of J1536+0441, discovered by Boroson+09)
    - https://ui.adsabs.harvard.edu/abs/2009ApJ...699L..22W/abstract
    - The authors present new VLA imaging of the candidate binary system J1536 originally identified by Boroson+2009. The VLA maps at 8.5 GHz reveal two sources, separated by 0.97 arcsec (5.1kpc in their adopted cosmology). While Wrobel+ do not come to any definitive conclusions, the data would support either the binary scenario, where the binary is separated by 0.1 pc, or a scenario in which the system represents a binary quasar systems separated by 5.1 kpc (in their adopted cosmology). Further data is needed to constrain which scenario is the correct one. 

- Comerford+2009 (Dual AGN in COSMOS,COSMOS J100043.15+020637.2) --> Reviewed 
    - https://ui.adsabs.harvard.edu/abs/2009ApJ...702L..82C/abstract
    - This dual system was originally identified in HST imaging as a merger that contained two distinct nuclei. Follow-up Keck spectroscopic of the two nuclei shows that both regions exhibit spectroscopic emission line ratios indicative of AGN excitation. The spectra of this system obtained for zCOSMOS agrees with this interpretation. Multiwavelength observations, while not spatially resolved, argues in favor of at least a single AGN in this system. The SED of the object is broadly consistent with a QSO that is dust obscured in the optical and UV. This system resides at z=0.36

- Lauer+2009 (HST imaging of J1536+0441) --> Skimmed 8 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2009ApJ...703..930L/abstract
    - Using HST and high S/N spectroscopy, the authors revisit the object J1536. The new data argue against the superposition (a new companion appears in imaging but it is argued that it is unlikely to be associated with the system at hand), and the argument of a recoiling/ejected SMBH is also disfavored. The binary hypothesis is not advanced with the new data, but it is not ruled out either. One possibility that arises is that this system could consistute a member of the double-peaked emitter class. No definitive conclusions are brought forward for this object.

- Decarli+2009 (Follow-up on J1536+0441) --> Skimmed 8 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2009ApJ...703L..76D/abstract
    - Authors identify in VLT imaging the quasar and its host, along with an elliptical companion. They claim that the quasar and companion positions coincide with the positions of the radio emission observed by Wrobel+2009, indicating that both host an AGN. They also suggest that the quasar resides ina. moderately rich cluster of galaxies. They argue that the data does seem to favor the superposition scenario, although if the campanion AGN emission obscured (given its redder color and faintness), then the double srt of broad lines would support instead the existence of the binary or a double-peaked emitter scenario in the quasar. Conclusion from Lauer+2009 clash with some of these conclusions.

- Dotti+2009 (Follow-up on J0927; argues for binary)
    - https://ui.adsabs.harvard.edu/abs/2009MNRAS.398L..73D/abstract
    - 


- Letawe+2009 (A 2nd AGN in HE 0450−2958) --> Read 3 August 2020 / 23 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2009MNRAS.396...78L/abstract
    - Whereas Magain+ and Haehnelt+ (and others) focused on the nature of the QSO, this paper focuses on the companion and used imfrared and optical imaging. The most intense infrared emission within the companion comes from a region which appears as a hole in the optical. Thus, rather than there being a 'hole' in the optical, there is instead of large cloud of dust roughly 5kpc in length. Deconvolution of the imaging suggests there is a point source lying behindor within the cloud of dust within this companion. The authors rule out the explanation that this point source arises from simply a compact hot cloud of dust. They also rule out the explanation that it arises due to a compact SB region strongly reddened by dust, as it would require some extreme conditions of SF which is not expected in a system like this. (Such extreme brightness of a source arising from SB occurs only in massive galaxies in rich clusters or in quasars, but wouldn't expected in a galaxy like this.) The interpretation that this is a bright AGN actually agrees well with the magnitudes observed for this point source. SED fitting also suggests that an AGN strongly obscured by dust is compatible with the non-detection in the optical and mid-IR. Thus the interprtation that the ULIRG hosts a dust reddened AGN, which manifests as the bright point source within/behind the dust cloud, stands as the best fitting explanation to the data. 
        - Original notes: Drawn from a survey of bright low-redshift QSOs. Bright QSO with no discernable host. Companion ULIRG with a 'hole' in the optical; best explained by a buried AGN which manifests as a compact bright spot in the NIR. Possible dual AGN candidate... also candidate for ejected BH as a result of three body interactions with a binary.

- Tang+2009 (Follow-up on J1536+0441)
    - https://ui.adsabs.harvard.edu/abs/2009ApJ...704.1189T/abstract
    - The authors propose that J1536 is both a double-peaked emitter AND and SMBH binary. In this scenario, the extra flux in the blue peaks of the broad lines comes from emission from the secondary SMBH, and the disk lines originate from the accretion disk of the primary SMBH. If this system were simply a double-peaked emitter, the authors claim that it would have to be a highly unusual one, in which the accretion disk is highly asymmetric. They also include comments on Arp 102B and 3C 390.3 in their discussion section. 
        - PROBABLY NEED TO DOUBLE CHECK THE DISCUSSION OF ARP 102B AND 3C 390.3.

- Xu+2009 (Double peaked in SDSS J131642.90+175332.5) --> Reviewed  / 9 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2009ApJ...705L..20X/abstract
    - All of the narrow emission lines in J1316 are double peaked, and an additional broad line component is observed in the forbidden lines and is found peaking in between the two narrow line systems. J1316 represents a good candidate dual AGN / binary AGN (not sure which one, actually...), but other physical phenomenon could explain the observed features. For now we'll consider it a dual candidate. 

- Wang+2009 (Double-peaked catalog from SDSS) --> Reviewed 29 June / 21 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2009ApJ...705L..76W/abstract
    - Wang used SDSS 7 and the MPA-JHU catalog to first select only AGN galaxies based on the BPT diagram, where nearly all of them lie above the Kewley line, with one falling just below ain the composite region defined by the Kauffman and Kewley lines. They narrowed the sample by selecting only local z (z≤0.15) systems, with equivalent widths of OIII>5 and SDSS r-band mags r<17.7. They look for double-peaked optical systems by fitting two gaussians to the emission lines with components separated by ≥1Å and a flux ratio of (0.1, 10). "For these galaxies which have very large or small flux ratios, the double-peaked profiles are not convincing. Finally, we are left with 87 sources having a flux ratio in the range of (0.3, 3), 44 of which have high enough signal-to-noise ratio of the spectra to see similar double-peaked structures in other lines (e.g., O iii λ4959, Hβ, Hα, Nii, S ii)." 
    - We denote that these systems are dual AGN candidates in our catalog. 


- Shields+2009 (J105041.35+345631.3) --> Reviewed  / 9 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2009ApJ...707..936S/abstract
    - Peculiarities of the spectrum of J1050 was first noticed in the systematic search by Smith+2010 (which first appeared on arxiv in 2009). J1050 shows typical narrow emission lines of an AGN at the redshift of the stellar absorption but the broad H-beta line is blue shifted by 3500 km s-1, which is near the upper end of expected velocities of reoicling SMBHs. The H-alpha shows a similar blue shift. Intensity ratios of the narrow emission lines indicate AGN excitation. J1050 is not within a rich cluster, and the authors argue in generak against a superposition scenario. The unusual spectral characteristics make J1050 a good candidate for a recoil scenario (however with some reasons to be cautious) as well as a good candidate for a double-peaked emitter. No definitive conclusions can at the time be drawn as to whether it is a recoil event or a double-peaked emitter. 





**********
3 Sept. 2020
In the interest of time, I am going through and rapidly removing irrelevant papers from the DR1 list. However, many of these papers may be relevant to DR2. I have already deledt many over the past day that I likely did not make a note of here (unfortunately), but for the set of 21 papers that I am removing, here are the listings:

- Demarco+2005
    - https://ui.adsabs.harvard.edu/abs/2005A%26A...432..381D/abstract
    
- Lauer+2005 
    - https://ui.adsabs.harvard.edu/abs/2005AJ....129.2138L/abstract
    
- Gaynullina+2005
    - https://ui.adsabs.harvard.edu/abs/2005A%26A...440...53G/abstract

- Croft+2005
    - https://ui.adsabs.harvard.edu/abs/2005AJ....130..867C/abstract

- Rauch+2005
    - https://ui.adsabs.harvard.edu/abs/2005ApJ...632...58R/abstract
    
- Misawa+2006
    - https://ui.adsabs.harvard.edu/abs/2006AJ....131...34M/abstract
    
- Kochanek+2006
    - https://ui.adsabs.harvard.edu/abs/2006ApJ...637L..73K/abstract
    
- Kochanek+2006
    - https://ui.adsabs.harvard.edu/abs/2006ApJ...640...47K/abstract
    
- Coppolani+2006
    - https://ui.adsabs.harvard.edu/abs/2006MNRAS.370.1804C/abstract

- Petry+2006
    - https://ui.adsabs.harvard.edu/abs/2006AJ....132.2046P/abstract
    
- Eastman+2007
    - https://ui.adsabs.harvard.edu/abs/2007ApJ...664L...9E/abstract

- Boris+2007
    - https://ui.adsabs.harvard.edu/abs/2007ApJ...666..747B/abstract

- Goicoechea+2008
    - https://ui.adsabs.harvard.edu/abs/2008NewA...13..182G/abstract
    
- Casey+2008
    - https://ui.adsabs.harvard.edu/abs/2008AJ....136..181C/abstract

- Faure+2009
    - https://ui.adsabs.harvard.edu/abs/2009A%26A...496..361F/abstract
    
- Decarli+2009
    - https://ui.adsabs.harvard.edu/abs/2009MNRAS.396L..31D/abstract

- Shalyapin+2009
    - https://ui.adsabs.harvard.edu/abs/2009MNRAS.397.1982S/abstract
    
- Borthakur+2010
    - https://ui.adsabs.harvard.edu/abs/2010ApJ...713..131B/abstract

- Cappetta+2010
    - https://ui.adsabs.harvard.edu/abs/2010MNRAS.407.1290C/abstract
 
- Borthakur+2011 
    - https://ui.adsabs.harvard.edu/abs/2011ApJ...727...52B/abstract
   
- Cupani+2011   
    - https://ui.adsabs.harvard.edu/abs/2011AN....332..319C/abstract
    - This one probably isn't relevant at all

- Rogerson+2012 
    - https://ui.adsabs.harvard.edu/abs/2012MNRAS.421..971R/abstract
    - Probably not relevant at all
    
- Dhar+2012
    - https://ui.adsabs.harvard.edu/abs/2012MNRAS.427..204D/abstract
    - Probably not relevant at all
    
- Wagg+2014 
    - https://ui.adsabs.harvard.edu/abs/2014ApJ...783...71W/abstract
    - I don't think is relevant
**********
   
   
   
   
- Misawa+2006 (Examination of 2 quasar pairs and 1 quasar triplet) --> Skimmed 2 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2006AJ....131...34M/abstract
    - These pairs are separated by many arcminutes and are therefore not being included in DR1. Might be included in DR2. 

- Prochaska+2012 (projected pairs of background/foreground quasars) --> Skimmed 3 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2013ApJ...762L..19P/abstract
    - Looks like these are far enough apart that we don't need to include in DR1. But will probably include some of these in DR2. 
    
- Mezcua+2014 (Photometric Decomposition of Mergers in Disk Galaxies) --> Skimmed 3 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2014ApJ...784...16M/abstract
    - Mezcua+ flag 19 of their double nuclei systems as candidate sub-parsec binary AGNs simply because the nuclei are closely spaced together (<1 kpc). While this is interesting and I'd like to see what data has been published for these, I won't be including them simply because the nuclei are close together. They didn't discuss at all whether the nuclei were active and therefore it remains to be seen if there is even AGN activity in either nuclei of each pair. 
    
- Farina+2014 (The extent of the Mg II absorbing circumgalactic medium of quasars) --> Skimmed 3 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2014MNRAS.441..886F/abstract
    - Projected pairs but with <.5 arcmin separations. Keep in mind for DR2?
    

- Heintz+2016 (Projected Pair of QSOs Separated by 4.5 arcsec) --> Checked 3 Sept. 2020
    - https://ui.adsabs.harvard.edu/abs/2016AJ....152...13H/abstract
    - Two QSOs with discordant redshifts. Possibly include in DR2
    
    
    

### Papers on Dual AGNs (>~1kpc separation) as well as closer binaries using Double Peaked OIII as a Candidate Selector and/or spectroscopic emission line ratios

- *** Owen+1985 (Dual AGN: 3C 75) --> (Reviewed 5 August 2020)
    - https://ui.adsabs.harvard.edu/abs/1985ApJ...294L..85O/abstract
    - Published: July 1985

- *** Brotherton+1999 (Dual Quasar: FIRST J164311.3+315618) --> (Reviewed 4 August)
    - https://ui.adsabs.harvard.edu/abs/1999ApJ...514L..61B/abstract
    - Published: 18 February 1999 

- *** Peng+1999 (Quasar Pair Q1634+267A,B)
    - https://ui.adsabs.harvard.edu/abs/1999ApJ...524..572P/abstract
    - Published: October 1999

- *** Mortlock+1999 (Small catalog of quasar pairs)
    - https://ui.adsabs.harvard.edu/abs/1999MNRAS.309..836M/abstract
    - Published: November 1999

- *** Schneider+2000 (High z quasar pair SDSSp J143952.58-003359.2) --> (Reviewed 17 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2000AJ....120.2183S/abstract
    - Published: November 2000
    - Serendipitous discovery of a quasar pair at z=4.25, with redshift differences consistent with zero.
    
- *** Junkkarinen+2001 (Dual Quasar: LBQS 0103-2753) --> (Reviewed 31 July 2020)
    - https://ui.adsabs.harvard.edu/abs/2001ApJ...549L.155J/abstract
    - Published: 2001 March 10
    

- *** Muñoz+2001a (Lens or Binary Quasar in RX J0921+4529) --> (Reviewed 19 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2001ApJ...546..769M/abstract
    - Published: January 2001
    - Two quasars detected coincident with large X-ray detected cluster. Excess of galaxies in the field as demonstrated by R band imaging, which also reveal the two blue objects, the quasars. Spectra obtained for several nearby galaxies shows redshifts of ~0.32 whereas the quasars are at z~1.66 with separation ~6.9''. Redshift of putative lens galaxy observed between the two could not be measured, although it is observed in HST imaging and is a galaxy with an apparent spiral structure. After subtracting the quasars, there is a faint residual near component B (with a slight offset between B and the residual); while it is likely a faint cluster galaxy, they cannot rule out that it is the host galaxy of B, in which case this would be a binary system. However, they show that if this is the host galaxy, and it is a lensed system, they should have detected this feature in A very easily but they do not. Putative lensing spiral galaxy and cluster have photometric redshifts consistent with z~0.32. Conclude that this is likely a lensed quasar, but follow-up observations could solidify this conclusion.
    
- Muñoz+2001c (Definite lensed quasar in B1152+199)
    - https://ui.adsabs.harvard.edu/abs/2001ApJ...563L.107M/abstract
    - Published: December 2001
    - 
    
- *** Gregg+2002 (Binary quasar: SDSS J233646.2-010732.6) --> (Reviewed 3 August)
    - https://ui.adsabs.harvard.edu/abs/2002ApJ...573L..85G/abstract
    - Published: 12 June 2002
    - 
    
- *** Green+2002 (Likely binary: QSO Pair Q2345+007) --> (Reviewed 19 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2002ApJ...571..721G/abstract
    - Published: June 2002
    - COnclude that the X-ray data disfavors the lens scenario, despite the incredible similarity of the two QSO's optical spectra. No evidence is found for a single cluster lens or luminous lens galaxy. The closest clusters to this QSO pair are not massive enough at a distance of 220 h_20 kpc to produce the observed images. X-ray spectra of the two sources are claimed to be incompatible. Green+2002 conclude that this is likely a true binary quasar system.
    
- *** Impey+2002 (Dual Quasar: LBQS 0015+0239) --> (Reviewed 3 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2002ApJ...574..623I/abstract
    - Published: 2002 August 1
    - 

- *** Komossa+2003 (Dual AGN: NGC 6240) --> (Reviewed 31 July 2020)
    - https://ui.adsabs.harvard.edu/abs/2003ApJ...582L..15K/abstract
    - Published: 2 December 2002 (copyright 2003)

- *** Faure+2003 (Likely binary quasar: LBQS 1429-0053) --> (Reviewed 19 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2003A%26A...405..415F/abstract
    - Published: May 2003
    - Identified by Hewitt+1989 as a lens system, Faure+2003 uses VLT and HST data to search for lensing cluster or galaxy. Find no evidence for a luminous lens galaxy between the two quasars, find that the galaxy cluster possibly at z~1 (based on photometric redshifts) that is within 5" of the quasars could not alone explain the images of the quasars (and thus that cluster could not alone be a sufficient lens), and no evidence for a dark lens. Possible lensed host galaxy in component A which at it's luminosity would not be seen in B, but this may also be foreground blue galaxy or actually the host of A itself. Ruled as a likely true quasar pair rather than a lens.


- *** Aldcroft+2003 (possible lens or binary: UM 425A,B) --> (Reviewed 18 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2003ApJ...592..710A/abstract
    - Published: August 2003
    - Aldcroft finds evidence for and against BOTH the binary and lens scenarios. In support of the lens scenario: similar redshifts, colors, and spectral type for both components; anamolous brightness of UM 425A and faintness of UM 425B, and there is some X-ray and H band evidence for an intervening galaxy group. In support of the binary: the sources show significantly different intrinsic absorption; any putative lensing galaxy group is ~6 times X-ray underluminous for the required lensing mass; significantly different UV emission-line profiles; emission line equivalent widths consistent with baldwin effect for apparent luminosity differences. Also, no evidence for intervening lens galaxy near the fainter counterpart UM 425B.
    - They present similar findings to that of Mathur+2003 (see below).

- *** Mathur+2003 (Proposed Dual quasar: UM 425A,B) --> (Reviewed 5 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2003ApJ...589L...1M/abstract
    - Published: May 2003
    - Originally flagged by Meylan+1989 as a possible lens. Mathur+2003 propose it is a true pair. Aldcroft+2003 also published on this (see above).
    - Claim that diffuse X-ray emission is in fact associated with a cluster around UM 425A, rather than a foreground cluster. Find no cluster near the fainter counterpart, and diffuse emission appears closer to the primary counterpart. Even if there is a foreground cluster at z~0.5, it is not massive enough to cause the observed lens images of UM 425. They find the X-ray spectra to be incompatible with one another.

- *** Heidt+2003 (Dual quasar: PKS 0537-441) --> (Reviewed 4 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2003A%26A...406..565H/abstract
    - Published: August 2003
    - Argues against the lensing theory by other authors. Find extended OII emission, and evidence for a potential QSO in a companion galaxy, making this a binary QSO system (if true).


- *** Djorgovski+2003 (Quasar pair associated w/ SDSS 0338+0021) --> (Reviewed 18 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2003ApJ...596...67D/abstract
    - Published: October 2003
    - 
    
    
- Inada+2003 (Nature publication) (Lens: SDSS J1029+2623)
    - https://ui.adsabs.harvard.edu/abs/2003Natur.426..810I/abstract
    - Published: December 2003
    -
    
- Oguri+2004 (Lens: SDSS J1004+4112)
    - https://ui.adsabs.harvard.edu/abs/2004ApJ...605...78O/abstract
    - Published: April 2004
   
   
- *** Klamer+2004 (Radio galaxies J0321−455N and J0321−455S [ akin to 3C 75]) 
    - https://ui.adsabs.harvard.edu/abs/2004MNRAS.351..101K/abstract
    - Published: June 2004

- *** Ballo+2004 (Dual AGN: Arp 299, comprising IC 694 and NGC 3690) --> (Reviewed 16 July)
    - https://ui.adsabs.harvard.edu/abs/2004ApJ...600..634B/abstract
    - Published: 2004 January 10

- *** Zhou+2004 (Binary AGN: SDSS J104807.74+005543.5) --> (Reviewed 13 July)
    - https://ui.adsabs.harvard.edu/abs/2004ApJ...604L..33Z/abstract
    - Published: 26 February 2004
    - Selected via double peaked narrow optical lines 

- *** Maness+2004 (Tentative binary AGN: 0402+379) --> (Reviewed 5 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2004ApJ...602..123M/abstract
    - Published: February 2004
    - First proposed binary case for CSO 0402+379

- *** Stockton+2004 (Dual AGN/dual quasar: 3C 294)  --> (Reviewed 3 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2004ApJ...600..626S/abstract
    - Published: 2004 January 10
    -

- *** Miller+2004 (Lenses and/or binary quasars)
    - https://ui.adsabs.harvard.edu/abs/2004MNRAS.348..395M/abstract
    - Published: February 2004
    - 
    
- *** Green+2004 (Likely true binary quasar: HS 1216+5032 A,B) --> (Reviewed 18 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2004MNRAS.349.1261G/abstract
    - Published: April 2004
    - No lens system had been previously detected when this system was found in the Hamburg Quasar Survey (Hagen+1996), but UV HST data (Lopez, Hagen & Reimers 2000) found complex intervening C IV absorbers, suggesting this could be a dark lens. Green+2004 use optical and IR spectroscopy to show that there are significant differences in the profiles of the H-alpha emission (lack of narrow component in B), only B component shows strong BAL systems, the continuum shapes of the systems differ significantly (and this is bolstered by B-R colors), A is the brighter of the two in the optical and IR, but B is radio loud while A is radio quiet. Therefore, highly likely this is a true binary quasar pair.
    

- *** Guainazzi+2005 (Dual AGN: ESO509-IG066) --> (Reviewed 29 July 2020)
    - https://ui.adsabs.harvard.edu/abs/2005A%26A...429L...9G/abstract
    - Published: 2005 January
    - 

- Oguri+2005 (Two lensed quasars: SDSS J100128.61+502756.9 and SDSS J120629.65+433217.6) 
    - https://ui.adsabs.harvard.edu/abs/2005ApJ...622..106O/abstract
    - Published: March 2005
    -
    
    
- *** Pindor+2006 (Lens: SDSS J102111.02+491330.4, Binary: SDSS J1120+6711) --> (Reviewed 18 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2006AJ....131...41P/abstract
    - Published: January 2006
    - SDSS J1021+4913: Likely len; identical redshifts, spectral similarities, K' band imaging reveals evidence for a lens galaxy between the two images. System is well fit with a two component model for the quasar images and another component for the lens galaxy in the foreground. 
    - SDSS J1120+6711: likely binary; indistinguishable redshifts, but have significantly different SEDs, and optical and H' band imaging do not reveal any signs of a lens system (system is well fit with a two component model).
    - Table included at the end that includes a list of rejected lens candidates as well as two potential binary quasars.

- *** Hudson+2006 (Chandra view of 3C 75, aka Abell 400) --> (Reviewed 18 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2006A%26A...453..433H/abstract
    - Published: 16 June 2006
    - X-ray cores of the dual radio core system are easily resolved, along with X-ray emission from cluster. X-ray sources are not highly absorbed, and this work proposes that the two AGNs 3C 75 are likely in a bound, merging system (or possible merger remnant) rather than an unbound (i.e. projected) system.

- Hennawi+2006 (Quasars pairs) --> (Reviewed 17 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2006AJ....131....1H/abstract
    - Published: January 2006
    - Used lens selection, color selection, photometric selection, and spectroscopic selection to select/confirm pairs of binary quasars, as well as obtained follow-up spectroscopic observations to confirm pairs. Use SDSS and 2QZ quasar samples.
        - Find sample of 221 new quasar pairs for redshift range 0.5<=z<=3.0. 26 of these have proper separations < 50 h^-1 kpc.


- *** Rodriguez+2006 (Binary AGN: 0402+379) --> (Reviewed 5 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2006ApJ...646...49R/abstract
    - Published: July 2006
    - Follow-up to Maness+2004, who first proposed the binary case. 

- *** Gerke+2007 (Dual AGN: EGSD2 J142033.66+525917.5 in AEGIS/DEEP2) --> (Reviewed 13 July)
    - https://ui.adsabs.harvard.edu/abs/2007ApJ...660L..23G/abstract
    - Published: 6 April 2007
    - Double peaked OIII selected

- Inada+2008a (statistical lens sample from SDSS DR3) 
    - https://ui.adsabs.harvard.edu/abs/2008AJ....135..496I/abstract
    - Published: February 2008
    
- Inada+2008b (Lens: SDSS J1004+4112)
    - https://ui.adsabs.harvard.edu/abs/2008PASJ...60L..27I/abstract
    - Published: Ocotber 2008


- *** Bianchi+2008 (Dual AGN: Mrk 463) --> (Reviewed 3 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2008MNRAS.386..105B/abstract
    - Published: May 2008

- Evans+2008 (3C 321)
    - https://ui.adsabs.harvard.edu/abs/2008ApJ...675.1057E/abstract
    - Published: 2008 March 10

- *** Comerford+2009a (Dual AGN: DEEP2 Survey Search) --> (Reviewed 13 July)
    - https://ui.adsabs.harvard.edu/abs/2009ApJ...698..956C/abstract
    - Published: 2009 May 26
    - Selection based on double-peaked lines and velocity-offset lines (hence duals or offset)

- *** Comerford+2009b (Dual AGN: COSMOS J100043.15+020637.2) --> (Reviewed 13 July)
    - https://ui.adsabs.harvard.edu/abs/2009ApJ...702L..82C/abstract
    - Published: 17 August 2009
    - Selected based on HST imaging but EXHIBITS double peaked optical lines

- *** Xu & Komossa+2009 (Dual AGN candididate: SDSS J131642.90+175332.5) --> (Reviewed 16 July)
    - https://ui.adsabs.harvard.edu/abs/2009ApJ...705L..20X/abstract
    - Published: 8 October 2009

- *** Letawe+2009 (Dual quasar: HE0450−2958) --> (Reviewed 3 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2009MNRAS.396...78L/abstract
    - Published: 03 June 2009
    - Drawn from a survey of bright low-redshift QSOs. Bright QSO with no discernable host. Companion ULIRG with a 'hole' in the optical; best explained by a buried AGN which manifests as a compact bright spot in the NIR. Possible dual AGN candidate... also candidate for ejected BH as a result of three body interactions with a binary.
    
- *** Wang+2009 (Dual AGN: Systematic search of SDSS) --> (Reviewed 29 June)
    - https://ui.adsabs.harvard.edu/abs/2009ApJ...705L..76W/abstract
    - Published: 14 October 2009
    - Dual AGN (~1kpc sep) selected based on double peaked narrow emission lines

- Liu+2010a (Dual AGN: Systematic search of SDSS) --> (Reviewed 29 June)
    - https://ui.adsabs.harvard.edu/abs/2010ApJ...708..427L/abstract
    - Published: 2009 December 11 • © 2010
    - Dual AGN selected based on double peaked narrow emission lines

- Liu+2010b (Dual AGN: Follow-up of four double-peak selected sources) --> (Reviewed 15 July)
    - https://ui.adsabs.harvard.edu/abs/2010ApJ...715L..30L/abstract
    - Published: 2010 April 30
    - Follow-up from Liu+2010a.
    - Four dual AGN candidates followed-up w/ NIR and optical imaging. Strong case of duals for 3/4 to 4/4.

- Piconcelli+2010 (Dual AGN: IRAS 20210+1121) --> (Reviewed 29 July 2020)
    - https://ui.adsabs.harvard.edu/abs/2010ApJ...722L.147P/abstract
    - Published: 2010 September 28
    - Mismatch of conclusions between optical spectra and IR, as well as unusual features/proposed CT secondary AGN motivated these XMM observations. XMM shows 2 hard X-ray point sources coincident with galaxies. Analysis suggested two AGNs.


- Hennawi+2010 (Quasar pairs) --> (Reviewed 17 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2010ApJ...719.1672H/abstract
    - Published: 2010 August 3
    - Used color-selection and photometric redshift techniques, combed SDSS photometric data for binary quasars, which they confirmed via follow-up spectroscopy. Ongoing program, ~50% complete so far. 
        - Find 24 new high redshift (z~ 3-4) quasar pairs, recover 3 known pairs.

- Shen+2010 (Quasar pairs)
    - https://ui.adsabs.harvard.edu/abs/2010ApJ...719.1693S/abstract
    - Published: 2010 August 3
    - Follow-up to Hennawi+2010...
    
    
   
- Smith+2010 (Dual AGN: Systematic line ratio search of SDSS) --> (Reviewed 29 June)
    - https://ui.adsabs.harvard.edu/abs/2010ApJ...716..866S/abstract
    - Published: 2010 May 21
    - Dual AGN selected based on double peaked emission lines, with emphasis on Type I AGNs


- Rosario+2010 ()
    - https://ui.adsabs.harvard.edu/abs/2010ApJ...716..131R/abstract
    - Published: 2010 May 14
    - 

- Veron-Cetty+2010 Catalog of quasars
    - https://ui.adsabs.harvard.edu/abs/2010A%26A...518A..10V/abstract

- McGurk+2011 (Dual AGN: J0952+2552) --> (Reviewed 3 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2011ApJ...738L...2M/abstract
    - Published: 2011 August 5
    - Object followed-up from Rosario+2011, originally selected by Smith+2010 catalog.

- Farina+2001 (Study of 6 low-z quasar pairs)
    - https://ui.adsabs.harvard.edu/abs/2011MNRAS.415.3163F/abstract
    - Published: August 2011

- Rosario+2011 ()
    - https://ui.adsabs.harvard.edu/abs/2011ApJ...739...44R/abstract
    - Published: 2011 September 6
    - 

- Greene+2011 ()
    - https://ui.adsabs.harvard.edu/abs/2011ApJ...732....9G/abstract
    - Published: 2011 April 7
    -

- Fu+2011a (Follow-up of double-peaked dual candidates) --> (Reviewed 2 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2011ApJ...733..103F/abstract
    - Published: 2011 May 11
    - Follow-up NIR imaging of double-peaked sources from Wang, Liu, and Smith catalogs, to look for mergers/double nuclei systems. 
    
- Fu+2011b (SDSS J150243.1+111557) --> (Reviewed 29 July 2020)
    - https://ui.adsabs.harvard.edu/abs/2011ApJ...740L..44F/abstract
    - Published: 2011 September 29
    - 



- Comerford+2011 (SDSS J171544.05+600835.7) --> (Reviewed 13 July)
    - https://ui.adsabs.harvard.edu/abs/2011ApJ...737L..19C/abstract
    - Published: 2011 July 22
    - Double peaked OIII selected



- Liu+2011a (Candidate triple identified during systematic search)
    - Published: 
    - 
    -

- Liu+2011b (Systematic search of SDSS) --> (Reviewed 15 July)
    - https://ui.adsabs.harvard.edu/abs/2011ApJ...737..101L/abstract
    - Published: 2011 August 8
    - Systematic search for multi-AGN systems in SDSS based upon spectroscopic emission line ratios

- Shen+2011 (Follow-up analysis of double peaked emitters in Liu+2011b) --> (Reviewed 20 July)
    - https://ui.adsabs.harvard.edu/abs/2011ApJ...735...48S/abstract
    - Published: 2011 June 15
    - 


- Kunert-Bajraszewska+2011 (FIRST J164311.3+315618) --> (Reviewed 4 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2011ApJ...736..125K/abstract
    - Published: 2011 July 15
    - Paper provides information on disturbed radio morphology. The binary was originally discovered by Brotherton+1999. Some info on the morphology was given by other papers, including this one. 
    

- Ge+2012 (Larger systematic survey of SDSS) --> (Reviewed 29 June)
    - https://ui.adsabs.harvard.edu/abs/2012ApJS..201...31G/abstract
    - Published: 2012 August 2
    - All double peaked emitters (AGN and non-AGN sources)
    
    
    
- Mazzarella+2012 (Mrk 266)
    - https://ui.adsabs.harvard.edu/abs/2012AJ....144..125M/abstract
    - Published: 2012 September 24
    
- Shields+2012 (LBQS 0103-2753)
    - https://ui.adsabs.harvard.edu/abs/2012ApJ...744..151S/abstract
    - Published: 2011 December 22
    - Follow-up to Junkkarinen+2001
    
    
- Barrows+2012 (dual AGN in a quasar) --> (Reviewed 28/29 July)
    - https://ui.adsabs.harvard.edu/abs/2012ApJ...744....7B/abstract
    - Published: 2011 December 7
    - Double-peaked selected; exhaustive multiwavelength investigation
    
- Comerford+2012 (Follow-up spectroscopy of double-peaked candidates)--> (Reviewed 28/29 July)
    - https://ui.adsabs.harvard.edu/abs/2012ApJ...753...42C/abstract
    - Published: 2012 June 13
    - Sample drawn from Liu+2010, Wang+2009, and Smith+2010. Find a sample of 14 'best' candidates.
    
    
    
- Tadhunter+2012 (PKS 0347+05)
    - https://ui.adsabs.harvard.edu/abs/2012MNRAS.427.1603T/abstract
    - Published: December 2012
    
    
    
- Comerford+2013 (Duals and Offsets in AGES: 2 double-peaked AGNs, 5 offset AGNs) --> (Reviewed 29 July 2020)
    - https://ui.adsabs.harvard.edu/abs/2013ApJ...777...64C/abstract
    - Published: 2013 October 17
    - Sample drawn from AGN and Galaxy Evolution Survey. Find 2 cases of double peaked AGNs which are considered candidate duals, and 5 cases of offset AGNs.
    

- Bianchi+2013 (Follow-up on claimed triple, NGC 3341)
    - https://ui.adsabs.harvard.edu/abs/2013MNRAS.435.2335B/abstract
    - Published: 05 September 2013


    
- Shi+2014 (Systematic search for double peaked emitters in LAMOST DR1) --> (Reviewed 13 July)
    - https://ui.adsabs.harvard.edu/abs/2014RAA....14.1234S/abstract
    - Published: 29 April 2014
    - Along with regular double peaked emitters, discussion of duals included.
    

- Hennawi+2015 
    - https://ui.adsabs.harvard.edu/abs/2015Sci...348..779H/abstract
    - Published: 15 May 2015
    
    
- Wang+2019 (Systematic search for double peaked emitters in LAMOST DR4) --> (Reviewed 13 July)
    - https://ui.adsabs.harvard.edu/abs/2019MNRAS.482.1889W/abstract
    - Published: 22 October 2018
    - Along with regular double peaked emitters, discussion of duals included.



## Papers on (\lesssim1kpc) Binaries / Recoiling Systems (Particularly earlier, precursor papers) based on high velocity shifts of optical lines (not necessarily double peaked, but can include)


- Sillanpää+2018 (OJ 287)
    - 
    - 


- Bonning+2007 (Recoiling BH search, first systematic search) --> (Reviewed 17 August 2020)
    - https://ui.adsabs.harvard.edu/abs/2007ApJ...666L..13B/abstract
    - Published: 14 August 2007
    - First systematic search for recoiling BHs. Do not find any convincing cases of such objects. See Eracleous for discussion on this paper.


- Komossa+2008 (Recoiling SMBH in J092712.65+294344.0) --> (Reviewed on 14 July)
    - https://ui.adsabs.harvard.edu/abs/2008ApJ...678L..81K/abstract
    - Published: 2008 May 10
    - But see the following: 
        - Shields+2009a: https://ui.adsabs.harvard.edu/abs/2009ApJ...696.1367S/abstract
        - Heckman+2009: https://ui.adsabs.harvard.edu/abs/2009ApJ...695..363H/abstract
        - Bogdanovic+2009: https://ui.adsabs.harvard.edu/abs/2009ApJ...697..288B/abstract
        - Vivek+2009: https://ui.adsabs.harvard.edu/abs/2009MNRAS.400L...6V/abstract
        - Dotti+2009: https://ui.adsabs.harvard.edu/abs/2009MNRAS.398L..73D/abstract
        - Decarli+2014: https://ui.adsabs.harvard.edu/abs/2014MNRAS.445.1558D/abstract

-

- Boroson+2009 (SDSS J153636.22+044127.0) --> (Reviewed on 14 July)
    - https://ui.adsabs.harvard.edu/abs/2009Natur.458...53B/abstract
    - Published: 05 March 2009
        - Need to check for additional papers on this target (as of July 14)

- Shields+2009b (SDSS J105041.35+345631.3) --> (Reviewed on 14 July)
    - https://ui.adsabs.harvard.edu/abs/2009ApJ...707..936S/abstract
    - Published: 2009 November 30

- Tang & Grindlay 2009 (SDSS J105041.35+345631.3) --> (Reviewed 16 July)
    - https://ui.adsabs.harvard.edu/abs/2009ApJ...704.1189T/abstract
    - Published: 2009 September 30
    
- Civano+2010 (CID-42) 
    - https://ui.adsabs.harvard.edu/abs/2010ApJ...717..209C/abstract
    - Published: July 2010
    - This target was also published by Comerford+2009 the COSMOS target, COSMOS J100043.15+020637.2)

- Decarli+2010a (4C+22.25, or J1000+2233 from Tsalmantza+2011) --> (Reviewed 16 July)
    - https://ui.adsabs.harvard.edu/abs/2010ApJ...720L..93D/abstract
    - Published: 2010 August 13
    - Identified via extreme velocity offsets of broadlines

- Fabbiano+2011 (NGC3393) --> (Reviewed 29 July 2020)
    - https://ui.adsabs.harvard.edu/abs/2011Natur.477..431F/abstract
    - Published: 31 August 2011

- Barrows+2011 (J093201.60+031858.7) --> (Reviewed 16 July)
    - https://ui.adsabs.harvard.edu/abs/2011NewA...16..122B/abstract
    - Published: February 2011
    - Identified via extreme velocity offsets (blue-shifted) of broad lines

- Tsalmantza+2011 (Binary BH search, first systematic search for BINARIES) --> (Reviewed on 14 July)
    - Title: A Systematic Search for Massive Black Hole Binaries in the Sloan Digital Sky Survey Spectroscopic Sample
    - https://ui.adsabs.harvard.edu/abs/2011ApJ...738...20T/abstract
    - Published: 2011 August 9

- Eracleous+2012 (Systematic search for sub-parsec binaries) --> (Reviewed 27/28/29 July 2020)
    - https://ui.adsabs.harvard.edu/abs/2012ApJS..201...23E/abstract
    - Published: 2012 July 5
    - Selected from SDSS, includes follow-up observations from separate observatories

- Shen+2013 (Search for subparsec binaries based on velocity shifts of lines) --> (Reviewed 28/29 July)
    - https://ui.adsabs.harvard.edu/abs/2013ApJ...775...49S/abstract
    - Published: 2013 September 3
    - Selected from SDSS and used multi-epoch SDSS data
    
- Barrows+2013 (Sample of double-peaked quasars at higher z, looking for duals) --> (Reviewed 28/29 July 2020)
    - https://ui.adsabs.harvard.edu/abs/2013ApJ...769...95B/abstract
    - Published: 2013 May 8
    - Selected based on double peaks in NeV and NeIII lines. First systematic search for multi-AGN at higher z.


- Ju+2013 (Search for binaries in SDSS)
    - https://ui.adsabs.harvard.edu/abs/2013ApJ...777...44J/abstract
    - Published: 2013 October 15
    - 

- Tsai+2013 (WISE J233237.05-505643.5)
    - https://ui.adsabs.harvard.edu/abs/2013ApJ...779...41T/abstract
    - Published: 2013 November 25
        - Need to check for additional papers on this target (as of July 14)
        
- F. Liu+2014 (J120136.02+300305.5)
    - https://ui.adsabs.harvard.edu/abs/2014ApJ...786..103L/abstract
    - 
        
- X. Liu+2014 (Follow-up to Shen+2013)
    - https://ui.adsabs.harvard.edu/abs/2014ApJ...789..140L/abstract
    - 
    
     
- Graham+2015a (PG 1302-102) --> (Reviewed on 14 July)
    - Title: A possible close supermassive black-hole binary in a quasar with optical periodicity
    - https://ui.adsabs.harvard.edu/abs/2015Natur.518...74G/abstract
    
    
    
    
- Graham+2015b
    - Title: A systematic search for close supermassive black hole binaries in the Catalina Real-time Transient Survey
    - https://ui.adsabs.harvard.edu/abs/2015MNRAS.453.1562G/abstract

## Triple AGNs / Triple AGN Candidates / Triplet Quasars from the literature:
---------

Check:
- Pfeifle+2019b,2019a

- Liu+2019

- Liu+2011

- Koss+

- Kalfountzou+2017

- Djorgovsky+

- Farina+2013

- Deane+2014, but then immediately Wrobel+2014

- That one paper I found recently... Nyland is on it. Lacy 2019?

- Barth+2008 and then the Bianchi paper...

- Owen+ ? (Or was this one by Ge?)



In [3]:
wang2009 = (Table.read('wang2009/table1.dat', readme = 'wang2009/ReadMe', format='ascii.cds')).to_pandas()

liu2010 = (Table.read('xliu2010/table1.dat', readme = 'xliu2010/ReadMe', format='ascii.cds')).to_pandas()

# Loading in Smith+2010 catalog of double-peaked [OIII] emission line AGNs here. These include Type I and Type II.
# We're only interested in table 1 from the paper (the full list). Tables 2 and 3 have [OIII] and radio flux/luminosity information for subsets
smith2010t1 = (Table.read('smith2010/table1.dat', readme = 'smith2010/ReadMe', format='ascii.cds')).to_pandas()
#smith2010t2 = (Table.read('smith2010/table2.dat', readme = 'smith2010/ReadMe', format='ascii.cds')).to_pandas()
#smith2010t3 = (Table.read('smith2010/table3.dat', readme = 'smith2010/ReadMe', format='ascii.cds')).to_pandas()

# Loading in the Ge+2012 catalog of double-peaked [OIII] emission line galaxies here. 
# These include all double-peaked galaxies, irrespective of their BPT classifications.
ge2012t3 = (Table.read('Ge2012/table3.dat', readme = 'Ge2012/ReadMe', format='ascii.cds')).to_pandas()
ge2012t4 = (Table.read('Ge2012/table4.dat', readme = 'Ge2012/ReadMe', format='ascii.cds')).to_pandas()
ge2012t5 = (Table.read('Ge2012/table5.dat', readme = 'Ge2012/ReadMe', format='ascii.cds')).to_pandas()
ge2012t6 = (Table.read('Ge2012/table6.dat', readme = 'Ge2012/ReadMe', format='ascii.cds')).to_pandas()
ge2012t7 = (Table.read('Ge2012/table7.dat', readme = 'Ge2012/ReadMe', format='ascii.cds')).to_pandas()


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'sigma' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'VHb1' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'VHb2' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'MJD2' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'Plate2' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/anaconda3/l

In [6]:
#ge2012t5

In [7]:
#ge2012t6

In [8]:
#smith2010t3

In [9]:
#ge2012t3

In [112]:
#ge2012 = (pd.concat([ge2012]*2, ignore_index=True)).sort_values(by=ge2012['SDSS'], ascending=True, axis=1)

ge2012=pd.concat([ge2012]*2).sort_index().reset_index(drop=True) # Duplicating every row so that we have 2 lines per dual AGN candidate

In [10]:
#ge2012

In [104]:
def name_to_coords(df,dfcol):
    # A function to take in the SDSS designation in a form JXXXXXX.XX+XXXXXX.X and convert this to sexagesimal coordinates, where the '+' could be '+' or '-'.
    # Output form will be: XX:XX:XX.XX +XX:XX:XX.X, where the '+' could be '+' or '-'
    df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
    df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
    df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
    df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
    df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
    df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
    return

In [105]:
name_to_coords(ge2012,ge2012['SDSS'])

In [11]:
#ge2012